In [1]:
!pip install ultralytics > /dev/null

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from ultralytics import YOLO
from ultralytics.nn.modules import Conv, C2f, SPPF, Detect
from ultralytics.utils import LOGGER
from ultralytics.models.yolo.detect import DetectionTrainer
from ultralytics.utils.torch_utils import select_device, smart_inference_mode
import numpy as np
from typing import Dict, List, Tuple, Optional, Any, Union
import math
import os
import sys
from pathlib import Path
import yaml
import shutil
from sklearn.model_selection import train_test_split
from torch.optim import AdamW, SGD
import warnings
import logging
import gc
import time
from scipy import ndimage
import cv2
from PIL import Image
import matplotlib
matplotlib.use('Agg')  # Использовать non-interactive backend
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
# Специальные фильтры для matplotlib warnings
warnings.filterwarnings('ignore', category=RuntimeWarning, module='matplotlib')
warnings.filterwarnings('ignore', message='invalid value encountered in less')
warnings.filterwarnings('ignore', message='invalid value encountered in greater')
warnings.filterwarnings('ignore', message='divide by zero encountered')
# Настройка matplotlib для избежания warnings
plt.ioff()  # Отключить интерактивный режим

# Импорты для метрик из metric.py
import json
import random
import pandas as pd
from numba import jit
from concurrent.futures import ThreadPoolExecutor

# Глобальная переменная для режима отладки
# Если True - обучение на 800 изображениях, минимум 10 эпох, валидация на всех данных
# Если False - обучение на всех данных
IS_DEBUG = False

# Пути к датасетам Kaggle
# Определяем пути к датасетам в зависимости от среды
if os.path.exists('/kaggle/input/'):
    # Kaggle окружение
    TRAIN_DATASET_1 = '/kaggle/input/01trains1datasethumanrescu1'
    TRAIN_DATASET_2 = '/kaggle/input/02secondpartdatasethumanrescue'
    VAL_DATASET_PUBLIC = '/kaggle/input/03validationdatasethumanrescue/public'
    VAL_DATASET_PRIVATE = '/kaggle/input/03validationdatasethumanrescue/private'
else:
    # Локальная среда - создаем тестовые пути
    TRAIN_DATASET_1 = './datasets/train1'
    TRAIN_DATASET_2 = './datasets/train2'
    VAL_DATASET_PUBLIC = './datasets/val_public'
    VAL_DATASET_PRIVATE = './datasets/val_private'
    # Создаем директории если их нет
    for dataset_path in [TRAIN_DATASET_1, TRAIN_DATASET_2, VAL_DATASET_PUBLIC, VAL_DATASET_PRIVATE]:
        os.makedirs(dataset_path, exist_ok=True)

# Определяем директорию для логов в зависимости от среды
if os.path.exists("/kaggle/working/"):
    log_dir = "/kaggle/working/"
else:
    log_dir = "./logs/"
    os.makedirs(log_dir, exist_ok=True)

logger = logging.getLogger('ml')
logger.setLevel(logging.DEBUG)

# Prevent adding handlers multiple times
if logger.hasHandlers():
    logger.handlers.clear()

# File handlers for different log levels
from logging.handlers import RotatingFileHandler

# Error log handler
error_handler = RotatingFileHandler(
    os.path.join(log_dir, 'main_error.log'),
    maxBytes=1*1024*1024,  # 1MB
    backupCount=5,
    encoding='utf-8'
)
error_handler.setLevel(logging.ERROR)

# Warning log handler
warning_handler = RotatingFileHandler(
    os.path.join(log_dir, 'main_warning.log'),
    maxBytes=1*1024*1024,  # 1MB
    backupCount=5,
    encoding='utf-8'
)
warning_handler.setLevel(logging.WARNING)

# Debug log handler
debug_handler = RotatingFileHandler(
    os.path.join(log_dir, 'main_debug.log'),
    maxBytes=1*1024*1024,  # 1MB
    backupCount=5,
    encoding='utf-8'
)
debug_handler.setLevel(logging.DEBUG)

# Console handler
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)

# Formatter
formatter = logging.Formatter(
    '%(asctime)s - MAIN - %(levelname)s - %(funcName)s:%(lineno)d - %(message)s'
)
error_handler.setFormatter(formatter)
warning_handler.setFormatter(formatter)
debug_handler.setFormatter(formatter)
console_handler.setFormatter(formatter)

# Add handlers
logger.addHandler(error_handler)
logger.addHandler(warning_handler)
logger.addHandler(debug_handler)
logger.addHandler(console_handler)

logger.info("Logging setup completed")
print("[DEBUG] Logging setup completed successfully.")

def validate_dataset_annotations(dataset_path: str, max_check: int = 50) -> Dict:
    """Валидация аннотаций датасета для проверки корректности формата YOLO"""
    validation_results = {
        'valid_annotations': 0,
        'invalid_annotations': 0,
        'empty_annotations': 0,
        'class_distribution': {},
        'coordinate_errors': 0,
        'format_errors': 0,
        'sample_annotations': []
    }
    
    labels_dir = os.path.join(dataset_path, 'labels')
    if not os.path.exists(labels_dir):
        logger.warning(f"⚠️ Директория меток не найдена: {labels_dir}")
        return validation_results
    
    # Получение файлов аннотаций
    label_files = [f for f in os.listdir(labels_dir) if f.endswith('.txt')]
    check_files = label_files[:max_check] if len(label_files) > max_check else label_files
    
    logger.info(f"🔍 Проверка {len(check_files)} файлов аннотаций из {len(label_files)}")
    
    for label_file in check_files:
        label_path = os.path.join(labels_dir, label_file)
        
        try:
            with open(label_path, 'r') as f:
                lines = f.readlines()
            
            if not lines or all(line.strip() == '' for line in lines):
                validation_results['empty_annotations'] += 1
                continue
            
            file_valid = True
            for line_num, line in enumerate(lines, 1):
                line = line.strip()
                if not line:
                    continue
                
                parts = line.split()
                if len(parts) != 5:
                    logger.warning(f"⚠️ {label_file}:{line_num} - Неверное количество значений: {len(parts)} (ожидается 5)")
                    validation_results['format_errors'] += 1
                    file_valid = False
                    continue
                
                try:
                    class_id = int(parts[0])
                    x_center, y_center, width, height = map(float, parts[1:5])
                    
                    # Проверка диапазона координат (должны быть 0-1)
                    if not (0 <= x_center <= 1 and 0 <= y_center <= 1 and 
                           0 <= width <= 1 and 0 <= height <= 1):
                        logger.warning(f"⚠️ {label_file}:{line_num} - Координаты вне диапазона [0,1]: {parts[1:5]}")
                        validation_results['coordinate_errors'] += 1
                        file_valid = False
                    
                    # Подсчет классов
                    validation_results['class_distribution'][class_id] = \
                        validation_results['class_distribution'].get(class_id, 0) + 1
                    
                except ValueError as e:
                    logger.warning(f"⚠️ {label_file}:{line_num} - Ошибка преобразования: {e}")
                    validation_results['format_errors'] += 1
                    file_valid = False
            
            if file_valid:
                validation_results['valid_annotations'] += 1
                # Сохранение примера валидной аннотации
                if len(validation_results['sample_annotations']) < 3:
                    validation_results['sample_annotations'].append({
                        'file': label_file,
                        'content': lines[:3]  # Первые 3 строки
                    })
            else:
                validation_results['invalid_annotations'] += 1
                
        except Exception as e:
            logger.error(f"❌ Ошибка чтения {label_file}: {e}")
            validation_results['invalid_annotations'] += 1
    
    # Логирование результатов
    logger.info(f"📊 Результаты валидации аннотаций:")
    logger.info(f"   ✅ Валидные: {validation_results['valid_annotations']}")
    logger.info(f"   ❌ Невалидные: {validation_results['invalid_annotations']}")
    logger.info(f"   📄 Пустые: {validation_results['empty_annotations']}")
    logger.info(f"   🔢 Ошибки формата: {validation_results['format_errors']}")
    logger.info(f"   📍 Ошибки координат: {validation_results['coordinate_errors']}")
    logger.info(f"   🏷️ Распределение классов: {validation_results['class_distribution']}")
    
    return validation_results

def create_combined_dataset_yaml(output_path: str = 'combined_data.yaml') -> str:
    """Создание YAML конфигурации для объединенных датасетов"""
    
    # Проверка структуры валидационного датасета
    val_images_path = os.path.join(VAL_DATASET_PUBLIC, 'images')
    val_labels_path = os.path.join(VAL_DATASET_PUBLIC, 'labels')
    
    if os.path.exists(VAL_DATASET_PUBLIC):
        if not os.path.exists(val_images_path):
            logger.warning(f"⚠️ Папка images не найдена в валидационном датасете: {val_images_path}")
        if not os.path.exists(val_labels_path):
            logger.warning(f"⚠️ Папка labels не найдена в валидационном датасете: {val_labels_path}")
        
        if os.path.exists(val_images_path) and os.path.exists(val_labels_path):
            val_images_count = len([f for f in os.listdir(val_images_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
            val_labels_count = len([f for f in os.listdir(val_labels_path) if f.lower().endswith('.txt')])
            logger.info(f"📊 Валидационный датасет: {val_images_count} изображений, {val_labels_count} меток")
        else:
            logger.error(f"❌ Неправильная структура валидационного датасета: {VAL_DATASET_PUBLIC}")
    else:
        logger.warning(f"⚠️ Валидационный датасет не найден: {VAL_DATASET_PUBLIC}")
    
    # В режиме отладки создаем временную структуру
    if IS_DEBUG:
        logger.info("🐛 Создание YAML для debug режима")
        
        # Создание debug структуры
        train_datasets = [TRAIN_DATASET_1, TRAIN_DATASET_2]
        debug_train_dir = create_debug_dataset_structure(train_datasets, 800)
        
        if debug_train_dir:
            # Определение базового пути для debug датасета
            if os.path.exists('/kaggle/working/'):
                debug_base_path = '/kaggle/working/debug_dataset'
            else:
                debug_base_path = './debug_dataset'
            
            # Валидация аннотаций в debug датасете
            debug_dataset_path = os.path.join(debug_base_path, 'train')
            validation_results = validate_dataset_annotations(debug_dataset_path)
            logger.info(f"📊 Результаты валидации аннотаций: {validation_results}")
            
            # Дополнительная диагностика аннотаций
            labels_dir = os.path.join(debug_dataset_path, 'labels')
            if os.path.exists(labels_dir):
                label_files = [f for f in os.listdir(labels_dir) if f.endswith('.txt')]
                logger.info(f"📁 Найдено файлов аннотаций: {len(label_files)}")
                
                # Проверим несколько файлов аннотаций
                sample_files = label_files[:5] if len(label_files) >= 5 else label_files
                for label_file in sample_files:
                    label_path = os.path.join(labels_dir, label_file)
                    try:
                        with open(label_path, 'r') as f:
                            lines = f.readlines()
                            if lines:
                                logger.info(f"📄 {label_file}: {len(lines)} объектов, первая строка: {lines[0].strip()}")
                            else:
                                logger.info(f"📄 {label_file}: пустой файл")
                    except Exception as e:
                        logger.warning(f"⚠️ Ошибка чтения {label_file}: {e}")
            else:
                logger.warning(f"⚠️ Папка с аннотациями не найдена: {labels_dir}")
            
            yaml_config = {
                'path': debug_base_path,
                'train': os.path.join(debug_dataset_path, 'images'),
                'val': os.path.join(VAL_DATASET_PUBLIC, 'images') if os.path.exists(VAL_DATASET_PUBLIC) else '',
                'test': VAL_DATASET_PRIVATE if os.path.exists(VAL_DATASET_PRIVATE) else '',
                'nc': 1,
                'names': ['human']
            }
            
            # Сохранение YAML файла
            try:
                with open(output_path, 'w', encoding='utf-8') as f:
                    yaml.dump(yaml_config, f, default_flow_style=False, allow_unicode=True)
                logger.info(f"✅ Debug YAML конфигурация сохранена: {output_path}")
                logger.info(f"   Обучение: {yaml_config['train']}")
                logger.info(f"   Валидация: {yaml_config['val']}")
                return output_path
            except Exception as e:
                logger.error(f"❌ Ошибка сохранения debug YAML: {e}")
                return ''
    
    # Обычный режим - создание YAML конфигурации без избыточного анализа
    logger.info("📝 Создание YAML конфигурации для полного режима обучения")
    
    # Создание конфигурации YAML
    yaml_config = {
        'path': os.path.dirname(os.path.abspath(output_path)),
        'train': [],
        'val': os.path.join(VAL_DATASET_PUBLIC, 'images') if os.path.exists(VAL_DATASET_PUBLIC) else '',
        'test': VAL_DATASET_PRIVATE if os.path.exists(VAL_DATASET_PRIVATE) else '',
        'nc': 1,
        'names': ['human']
    }
    
    # Добавление обучающих датасетов
    if os.path.exists(TRAIN_DATASET_1):
        yaml_config['train'].append(os.path.join(TRAIN_DATASET_1, 'images'))
    if os.path.exists(TRAIN_DATASET_2):
        yaml_config['train'].append(os.path.join(TRAIN_DATASET_2, 'images'))
    
    # Если только один датасет, убираем список
    if len(yaml_config['train']) == 1:
        yaml_config['train'] = yaml_config['train'][0]
    elif len(yaml_config['train']) == 0:
        logger.error("❌ Не найдено ни одного обучающего датасета!")
        yaml_config['train'] = ''
    
    # Сохранение YAML файла
    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            yaml.dump(yaml_config, f, default_flow_style=False, allow_unicode=True)
        logger.info(f"✅ YAML конфигурация сохранена: {output_path}")
        return output_path
    except Exception as e:
        logger.error(f"❌ Ошибка сохранения YAML: {e}")
        return ''

def collect_hierarchical_images(dataset_path: str) -> List[Tuple[str, str]]:
    """
    Сбор всех пар изображение-метка из иерархической структуры датасета
    
    Args:
        dataset_path: Путь к датасету с подпапками
        
    Returns:
        List[Tuple[str, str]]: Список пар (путь_к_изображению, путь_к_метке)
    """
    image_label_pairs = []
    
    images_base = os.path.join(dataset_path, 'images')
    labels_base = os.path.join(dataset_path, 'labels')
    
    if not (os.path.exists(images_base) and os.path.exists(labels_base)):
        logger.warning(f"⚠️ Не найдены папки images или labels в {dataset_path}")
        return image_label_pairs
    
    # Получение списка подпапок в images
    try:
        image_subdirs = [d for d in os.listdir(images_base) 
                        if os.path.isdir(os.path.join(images_base, d))]
        image_subdirs = sorted(image_subdirs)
        
        logger.info(f"📁 Найдено {len(image_subdirs)} подпапок в {images_base}: {image_subdirs}")
        
        for subdir in image_subdirs:
            images_subdir = os.path.join(images_base, subdir)
            labels_subdir = os.path.join(labels_base, subdir)
            
            if not os.path.exists(labels_subdir):
                logger.warning(f"⚠️ Не найдена соответствующая папка меток: {labels_subdir}")
                continue
            
            # Сбор изображений из подпапки
            try:
                image_files = [f for f in os.listdir(images_subdir) 
                             if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
                
                for img_file in image_files:
                    img_path = os.path.join(images_subdir, img_file)
                    
                    # Поиск соответствующей метки
                    label_name = os.path.splitext(img_file)[0] + '.txt'
                    label_path = os.path.join(labels_subdir, label_name)
                    
                    if os.path.exists(label_path):
                        image_label_pairs.append((img_path, label_path))
                    else:
                        logger.warning(f"⚠️ Не найдена метка для {img_file}: {label_path}")
                        
            except Exception as e:
                logger.error(f"❌ Ошибка обработки подпапки {subdir}: {e}")
                continue
        
        logger.info(f"✅ Собрано {len(image_label_pairs)} пар изображение-метка из {dataset_path}")
        
    except Exception as e:
        logger.error(f"❌ Ошибка сбора иерархических данных из {dataset_path}: {e}")
    
    return image_label_pairs

def create_debug_dataset_structure(train_datasets: List[str], limit: int = 800) -> str:
    """Создание временной структуры датасета для отладки с ограниченным количеством изображений"""
    if not IS_DEBUG:
        return None
    
    logger.info(f"🐛 Создание debug структуры с ограничением {limit} изображений")
    
    # Создание временных директорий - адаптация для локальной среды
    if os.path.exists('/kaggle/working/'):
        debug_base_dir = '/kaggle/working/debug_dataset'
    else:
        debug_base_dir = './debug_dataset'
        os.makedirs(debug_base_dir, exist_ok=True)
    
    debug_train_dir = os.path.join(debug_base_dir, 'train')
    debug_train_images = os.path.join(debug_train_dir, 'images')
    debug_train_labels = os.path.join(debug_train_dir, 'labels')
    
    # Очистка и создание директорий
    if os.path.exists(debug_base_dir):
        shutil.rmtree(debug_base_dir)
    
    os.makedirs(debug_train_images, exist_ok=True)
    os.makedirs(debug_train_labels, exist_ok=True)
    
    total_copied = 0
    
    # Копирование ограниченного количества файлов
    for dataset_path in train_datasets:
        if total_copied >= limit:
            break
            
        images_dir = os.path.join(dataset_path, 'images')
        labels_dir = os.path.join(dataset_path, 'labels')
        
        if not (os.path.exists(images_dir) and os.path.exists(labels_dir)):
            logger.warning(f"⚠️ Не найдены папки images/labels в {dataset_path}")
            continue
        
        # Проверка на иерархическую структуру (подпапки)
        try:
            subdirs = [d for d in os.listdir(images_dir) 
                      if os.path.isdir(os.path.join(images_dir, d))]
            
            if subdirs:  # Иерархическая структура (как TRAIN_DATASET_1)
                logger.info(f"📁 Обнаружена иерархическая структура в {dataset_path}")
                image_label_pairs = collect_hierarchical_images(dataset_path)
                
                # Ограничение количества пар для отладки
                selected_pairs = image_label_pairs[:limit - total_copied]
                
                for img_path, label_path in selected_pairs:
                    if total_copied >= limit:
                        break
                    
                    # Получение имени файла
                    img_filename = os.path.basename(img_path)
                    label_filename = os.path.basename(label_path)
                    
                    # Создание уникальных имен для отладочного датасета с сохранением соответствия
                    base_name = f"{total_copied:06d}_{os.path.splitext(img_filename)[0]}"
                    img_ext = os.path.splitext(img_filename)[1]
                    dst_img = os.path.join(debug_train_images, f"{base_name}{img_ext}")
                    dst_label = os.path.join(debug_train_labels, f"{base_name}.txt")
                    
                    # Копирование файлов
                    try:
                        shutil.copy2(img_path, dst_img)
                        shutil.copy2(label_path, dst_label)
                        total_copied += 1
                    except Exception as e:
                        logger.warning(f"⚠️ Ошибка копирования {img_path}: {e}")
                        continue
                        
            else:  # Плоская структура (старый код)
                logger.info(f"📄 Обнаружена плоская структура в {dataset_path}")
                
                # Получение списка изображений
                images = [f for f in os.listdir(images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
                images = sorted(images)[:min(limit - total_copied, len(images))]
                
                for img_file in images:
                    if total_copied >= limit:
                        break
                        
                    # Копирование изображения с сохранением соответствия имен
                    src_img = os.path.join(images_dir, img_file)
                    base_name = f"{total_copied:06d}_{os.path.splitext(img_file)[0]}"
                    img_ext = os.path.splitext(img_file)[1]
                    dst_img = os.path.join(debug_train_images, f"{base_name}{img_ext}")
                    
                    try:
                        shutil.copy2(src_img, dst_img)
                        
                        # Копирование соответствующей метки
                        label_file = os.path.splitext(img_file)[0] + '.txt'
                        src_label = os.path.join(labels_dir, label_file)
                        dst_label = os.path.join(debug_train_labels, f"{base_name}.txt")
                        
                        if os.path.exists(src_label):
                            shutil.copy2(src_label, dst_label)
                        
                        total_copied += 1
                        
                    except Exception as e:
                        logger.warning(f"⚠️ Ошибка копирования {img_file}: {e}")
                        continue
                        
        except Exception as e:
            logger.error(f"❌ Ошибка обработки датасета {dataset_path}: {e}")
            continue
    
    logger.info(f"✅ Создана debug структура: {total_copied} изображений в {debug_train_dir}")
    return debug_train_dir

def prepare_debug_dataset(train_paths: List[str], limit: int = 800) -> List[str]:
    """Подготовка ограниченного датасета для отладки"""
    if not IS_DEBUG:
        return train_paths
    
    logger.info(f"🐛 Режим отладки: ограничение до {limit} изображений")
    
    # Создание временной структуры
    debug_dir = create_debug_dataset_structure(train_paths, limit)
    if debug_dir:
        return [debug_dir]
    
    # Fallback к старой логике если создание структуры не удалось
    debug_train_paths = []
    total_images = 0
    
    for train_path in train_paths:
        if isinstance(train_path, str):
            images_dir = train_path
        else:
            continue
            
        if os.path.exists(images_dir):
            images = [f for f in os.listdir(images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            images = sorted(images)[:min(limit - total_images, len(images))]
            
            if images:
                debug_train_paths.append(images_dir)
                total_images += len(images)
                logger.info(f"📊 Добавлено {len(images)} изображений из {images_dir}")
            
            if total_images >= limit:
                break
    
    logger.info(f"🎯 Итого для отладки: {total_images} изображений")
    return debug_train_paths

def get_dataset_statistics(dataset_path: str) -> Dict:
    """Получение статистики датасета с поддержкой иерархической структуры"""
    stats = {
        'images_count': 0,
        'labels_count': 0,
        'classes_distribution': {},
        'image_sizes': [],
        'exists': False,
        'structure_type': 'unknown'
    }
    
    if not os.path.exists(dataset_path):
        return stats
    
    stats['exists'] = True
    images_dir = os.path.join(dataset_path, 'images')
    labels_dir = os.path.join(dataset_path, 'labels')
    
    if not (os.path.exists(images_dir) and os.path.exists(labels_dir)):
        return stats
    
    # Проверка на иерархическую структуру
    try:
        subdirs = [d for d in os.listdir(images_dir) 
                  if os.path.isdir(os.path.join(images_dir, d))]
        
        if subdirs:  # Иерархическая структура
            stats['structure_type'] = 'hierarchical'
            logger.info(f"📁 Анализ иерархической структуры в {dataset_path}")
            
            # Использование функции collect_hierarchical_images для подсчета
            image_label_pairs = collect_hierarchical_images(dataset_path)
            stats['images_count'] = len(image_label_pairs)
            stats['labels_count'] = len(image_label_pairs)
            
            # Анализ классов из меток
            class_counts = {}
            for _, label_path in image_label_pairs:
                try:
                    with open(label_path, 'r') as f:
                        lines = f.readlines()
                        for line in lines:
                            if line.strip():
                                class_id = int(line.split()[0])
                                class_counts[class_id] = class_counts.get(class_id, 0) + 1
                except Exception as e:
                    continue
            
            stats['classes_distribution'] = class_counts
            
        else:  # Плоская структура
            stats['structure_type'] = 'flat'
            logger.info(f"📄 Анализ плоской структуры в {dataset_path}")
            
            # Подсчет изображений
            try:
                image_files = [f for f in os.listdir(images_dir) 
                              if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
                stats['images_count'] = len(image_files)
                
                # Подсчет меток
                label_files = [f for f in os.listdir(labels_dir) 
                              if f.lower().endswith('.txt')]
                stats['labels_count'] = len(label_files)
                
                # Анализ классов
                class_counts = {}
                for label_file in label_files:
                    label_path = os.path.join(labels_dir, label_file)
                    try:
                        with open(label_path, 'r') as f:
                            lines = f.readlines()
                            for line in lines:
                                if line.strip():
                                    class_id = int(line.split()[0])
                                    class_counts[class_id] = class_counts.get(class_id, 0) + 1
                    except Exception as e:
                        continue
                
                stats['classes_distribution'] = class_counts
                
            except Exception as e:
                logger.error(f"❌ Ошибка анализа плоской структуры {dataset_path}: {e}")
                
    except Exception as e:
        logger.error(f"❌ Ошибка анализа структуры датасета {dataset_path}: {e}")
    
    return stats

def validate_on_private_dataset(model_path: str, private_dataset_path: str = VAL_DATASET_PRIVATE, debug_conf: float = None, debug_iou: float = None, debug_mode: bool = False) -> Dict:
    """Валидация модели на приватном датасете"""
    logger.info("🔒 Начало валидации на приватном датасете...")
    
    if not os.path.exists(model_path):
        logger.error(f"❌ Модель не найдена: {model_path}")
        return {'error': 'Model not found'}
    
    if not os.path.exists(private_dataset_path):
        logger.error(f"❌ Приватный датасет не найден: {private_dataset_path}")
        return {'error': 'Private dataset not found'}
    
    try:
        # Загрузка модели
        model = YOLO(model_path)
        logger.info(f"✅ Модель загружена: {model_path}")
        
        # Проверка структуры приватного датасета
        private_images_path = os.path.join(private_dataset_path, 'images')
        private_labels_path = os.path.join(private_dataset_path, 'labels')
        
        if not os.path.exists(private_images_path):
            logger.error(f"❌ Папка images не найдена в приватном датасете: {private_images_path}")
            return {'error': 'Private dataset images folder not found'}
        
        if not os.path.exists(private_labels_path):
            logger.error(f"❌ Папка labels не найдена в приватном датасете: {private_labels_path}")
            return {'error': 'Private dataset labels folder not found'}
        
        # Создание временного YAML для приватного датасета
        private_yaml_config = {
            'path': os.path.dirname(private_dataset_path),
            'train': os.path.join(TRAIN_DATASET_1, 'images') if os.path.exists(TRAIN_DATASET_1) else private_images_path,  # Добавляем обязательный 'train' ключ
            'val': private_images_path,  # Указываем на папку images
            'nc': 1,
            'names': ['human']
        }
        
        private_yaml_path = 'private_validation.yaml'
        with open(private_yaml_path, 'w', encoding='utf-8') as f:
            yaml.dump(private_yaml_config, f, default_flow_style=False, allow_unicode=True)
        
        # Валидация
        # Определение параметров валидации в зависимости от режима отладки
        val_conf = debug_conf if debug_mode and debug_conf is not None else 0.001
        val_iou = debug_iou if debug_mode and debug_iou is not None else 0.6
        
        logger.info(f"🔍 Выполнение валидации с параметрами: conf={val_conf}, iou={val_iou}...")
        val_results = model.val(
            data=private_yaml_path,
            imgsz=640,
            batch=16,
            conf=val_conf,  # Динамический порог уверенности
            iou=val_iou,    # Динамический IoU порог
            device='auto',
            plots=True,
            save_json=True,
            verbose=True
        )
        
        # Извлечение результатов
        results_dict = {
            'map50': float(val_results.box.map50) if hasattr(val_results, 'box') else 0.0,
            'map50_95': float(val_results.box.map) if hasattr(val_results, 'box') else 0.0,
            'precision': float(val_results.box.mp) if hasattr(val_results, 'box') else 0.0,
            'recall': float(val_results.box.mr) if hasattr(val_results, 'box') else 0.0,
            'f1_score': 0.0,
            'custom_metric_q': 0.0
        }
        
        # Вычисление F1-score
        if results_dict['precision'] > 0 and results_dict['recall'] > 0:
            results_dict['f1_score'] = 2 * (results_dict['precision'] * results_dict['recall']) / (results_dict['precision'] + results_dict['recall'])
        
        # Вычисление кастомной метрики Q
        logger.info("🎯 Вычисление кастомной метрики Q...")
        try:
            # Создание DataFrame с предсказаниями используя process_images_adapted
            predicted_df = process_images_adapted(private_dataset_path)
            
            if predicted_df is not None and len(predicted_df) > 0:
                logger.info(f"📊 Создан DataFrame с предсказаниями: {len(predicted_df)} записей")
                
                # Конвертация DataFrame в bytes для evaluate
                predicted_bytes = df_to_bytes(predicted_df)
                
                # Загрузка ground truth данных
                if os.path.exists(PUBLIC_GT_CSV_PATH):
                    gt_bytes = open_df_as_bytes(PUBLIC_GT_CSV_PATH)
                    
                    # Вычисление финальной метрики используя evaluate
                    custom_q = evaluate(
                        predicted_file=predicted_bytes,
                        gt_file=gt_bytes,
                        thresholds=np.round(np.arange(0.3, 1.0, 0.07), 2),
                        beta=1.0,
                        m=500,
                        parallelize=True
                    )
                    
                    if custom_q is not None and not np.isnan(custom_q) and not np.isinf(custom_q):
                        results_dict['custom_metric_q'] = custom_q
                        logger.info(f"✅ Кастомная метрика Q успешно вычислена: {custom_q:.4f}")
                    else:
                        logger.warning("⚠️ Получено некорректное значение кастомной метрики")
                        results_dict['custom_metric_q'] = 0.0
                else:
                    logger.warning(f"⚠️ Файл ground truth не найден: {PUBLIC_GT_CSV_PATH}")
                    results_dict['custom_metric_q'] = 0.0
            else:
                logger.warning("⚠️ Не удалось создать DataFrame с предсказаниями")
                results_dict['custom_metric_q'] = 0.0
        except Exception as e:
            logger.error(f"❌ Ошибка вычисления кастомной метрики: {e}")
            import traceback
            logger.error(f"❌ Полная трассировка ошибки: {traceback.format_exc()}")
            results_dict['custom_metric_q'] = 0.0
        
        logger.info("📊 Результаты валидации на приватном датасете:")
        logger.info(f"   mAP@0.5: {results_dict['map50']:.4f}")
        logger.info(f"   mAP@0.5:0.95: {results_dict['map50_95']:.4f}")
        logger.info(f"   Precision: {results_dict['precision']:.4f}")
        logger.info(f"   Recall: {results_dict['recall']:.4f}")
        logger.info(f"   F1-Score: {results_dict['f1_score']:.4f}")
        logger.info(f"   🎯 Кастомная метрика Q: {results_dict['custom_metric_q']:.4f}")
        
        # Очистка временного файла
        try:
            os.remove(private_yaml_path)
        except:
            pass
        
        return results_dict
        
    except Exception as e:
        logger.error(f"❌ Ошибка валидации на приватном датасете: {e}")
        import traceback
        traceback.print_exc()
        return {'error': str(e)}

def calculate_custom_metric(predictions: List[np.ndarray], ground_truths: List[np.ndarray], beta: float = 1.0) -> float:
    """
    Вычисление кастомной метрики согласно формуле (1)
    
    Args:
        predictions: Список предсказанных масок для каждого изображения
        ground_truths: Список истинных масок для каждого изображения  
        beta: Параметр для F-beta меры (по умолчанию 1.0)
        
    Returns:
        float: Значение кастомной метрики Q
    """
    # Проверка на None и пустые списки
    if predictions is None or ground_truths is None:
        logger.error("❌ Получены None значения для предсказаний или истинных масок")
        return 0.0
        
    if len(predictions) != len(ground_truths):
        logger.error(f"❌ Несоответствие количества предсказаний ({len(predictions)}) и истинных масок ({len(ground_truths)})")
        return 0.0
    
    if len(predictions) == 0:
        logger.warning("⚠️ Пустые списки предсказаний и истинных масок")
        return 0.0
    
    # Пороговые значения от 0.3 до 0.93 с шагом 0.07
    thresholds = np.arange(0.3, 0.94, 0.07)
    n_thresholds = len(thresholds)
    
    logger.info(f"📊 Вычисление кастомной метрики для {n_thresholds} порогов: {thresholds}")
    
    total_f_beta = 0.0
    valid_calculations = 0
    
    for threshold in thresholds:
        try:
            f_beta_t = calculate_f_beta_for_threshold(predictions, ground_truths, threshold, beta)
            # Проверка на None, NaN и Inf
            if f_beta_t is not None and not np.isnan(f_beta_t) and not np.isinf(f_beta_t):
                total_f_beta += f_beta_t
                valid_calculations += 1
                logger.info(f"   Порог {threshold:.2f}: F-beta = {f_beta_t:.4f}")
            else:
                logger.warning(f"⚠️ Некорректное значение F-beta для порога {threshold:.2f}: {f_beta_t}")
        except Exception as e:
            logger.error(f"❌ Ошибка вычисления F-beta для порога {threshold:.2f}: {e}")
    
    # Проверка на отсутствие валидных вычислений
    if valid_calculations == 0:
        logger.error("❌ Не удалось вычислить ни одного валидного значения F-beta")
        return 0.0
    
    # Финальная метрика - среднее арифметическое по валидным вычислениям
    try:
        custom_metric = total_f_beta / valid_calculations
        logger.info(f"🎯 Итоговая кастомная метрика Q = {custom_metric:.4f} (на основе {valid_calculations}/{n_thresholds} валидных вычислений)")
        return custom_metric
    except Exception as e:
        logger.error(f"❌ Ошибка вычисления финальной метрики: {e}")
        return 0.0


def calculate_f_beta_for_threshold(predictions: List[np.ndarray], ground_truths: List[np.ndarray], 
                                  threshold: float, beta: float = 1.0) -> float:
    """
    Вычисление F-beta меры для конкретного порога IoU
    
    Args:
        predictions: Список предсказанных масок
        ground_truths: Список истинных масок
        threshold: Пороговое значение IoU
        beta: Параметр для F-beta меры
        
    Returns:
        float: Значение F-beta меры для данного порога
    """
    total_tp = 0
    total_fp = 0
    total_fn = 0
    
    for pred_mask, gt_mask in zip(predictions, ground_truths):
        tp, fp, fn = calculate_tp_fp_fn_for_image(pred_mask, gt_mask, threshold)
        total_tp += tp
        total_fp += fp
        total_fn += fn
    
    # Вычисление F-beta меры по формуле (2)
    if total_tp + total_fp == 0 and total_tp + total_fn == 0:
        return 0.0
    
    if total_tp + total_fp == 0:
        precision = 0.0
    else:
        precision = total_tp / (total_tp + total_fp)
    
    if total_tp + total_fn == 0:
        recall = 0.0
    else:
        recall = total_tp / (total_tp + total_fn)
    
    if precision + recall == 0:
        return 0.0
    
    # F-beta формула с beta = 1
    f_beta = (1 + beta**2) * (precision * recall) / ((beta**2 * precision) + recall)
    
    return f_beta


def calculate_tp_fp_fn_for_image(pred_mask: np.ndarray, gt_mask: np.ndarray, threshold: float) -> Tuple[int, int, int]:
    """
    Вычисление TP, FP, FN для одного изображения согласно алгоритму
    
    Args:
        pred_mask: Предсказанная маска (2D массив с 0 и 1)
        gt_mask: Истинная маска (2D массив с 0 и 1)
        threshold: Пороговое значение IoU
        
    Returns:
        Tuple[int, int, int]: TP, FP, FN
    """
    # Извлечение областей предсказания и разметки
    pred_regions = extract_regions(pred_mask)
    gt_regions = extract_regions(gt_mask)
    
    if len(pred_regions) == 0 and len(gt_regions) == 0:
        return 0, 0, 0
    
    if len(pred_regions) == 0:
        return 0, 0, len(gt_regions)
    
    if len(gt_regions) == 0:
        return 0, len(pred_regions), 0
    
    # Создание матрицы соответствия IoU
    iou_matrix = create_iou_matrix(pred_regions, gt_regions)
    
    # Подсчет TP, FP, FN согласно алгоритму
    tp, fp, fn = count_tp_fp_fn_from_matrix(iou_matrix, threshold)
    
    return tp, fp, fn


def extract_regions(mask: np.ndarray) -> List[np.ndarray]:
    """
    Извлечение связных областей из бинарной маски
    
    Args:
        mask: Бинарная маска (2D массив с 0 и 1)
        
    Returns:
        List[np.ndarray]: Список масок для каждой связной области
    """
    # Поиск связных компонент
    labeled_mask, num_features = ndimage.label(mask)
    
    regions = []
    for i in range(1, num_features + 1):
        region_mask = (labeled_mask == i).astype(np.uint8)
        regions.append(region_mask)
    
    return regions


def get_model_predictions_as_masks(model, dataset_path: str, img_size: int = 640, conf_threshold: float = 0.25) -> Tuple[List[np.ndarray], List[np.ndarray]]:
    """
    Получение предсказаний модели в виде бинарных масок
    
    Args:
        model: Обученная модель YOLO
        dataset_path: Путь к датасету
        img_size: Размер изображения для инференса
        conf_threshold: Порог уверенности для детекций
        
    Returns:
        Tuple[List[np.ndarray], List[np.ndarray]]: Предсказанные маски и истинные маски
    """
    # Проверка входных параметров
    if model is None:
        logger.error("❌ Модель не предоставлена (None)")
        return [], []
    
    if not dataset_path or not os.path.exists(dataset_path):
        logger.error(f"❌ Путь к датасету не существует: {dataset_path}")
        return [], []
    
    logger.info(f"🔍 Анализ структуры датасета: {dataset_path}")
    logger.info(f"⚙️ Параметры: img_size={img_size}, conf_threshold={conf_threshold}")
    
    images_dir = os.path.join(dataset_path, 'images')
    labels_dir = os.path.join(dataset_path, 'labels')
    
    logger.info(f"📁 Папка изображений: {images_dir}")
    logger.info(f"🏷️ Папка меток: {labels_dir}")
    
    if not os.path.exists(images_dir) or not os.path.exists(labels_dir):
        logger.error(f"❌ Не найдены папки images или labels в {dataset_path}")
        logger.error(f"   images_dir exists: {os.path.exists(images_dir)}")
        logger.error(f"   labels_dir exists: {os.path.exists(labels_dir)}")
        return [], []
    
    # Проверяем структуру датасета
    try:
        subdirs = [d for d in os.listdir(images_dir) if os.path.isdir(os.path.join(images_dir, d))]
    except Exception as e:
        logger.error(f"❌ Ошибка чтения папки изображений: {e}")
        return [], []
    
    predictions = []
    ground_truths = []
    processed_count = 0
    error_count = 0
    
    if subdirs:  # Иерархическая структура
        logger.info(f"📁 Обнаружена иерархическая структура датасета с {len(subdirs)} подпапками: {subdirs}")
        image_label_pairs = collect_hierarchical_images(dataset_path)
        
        if not image_label_pairs:
            logger.error(f"❌ Не найдено пар изображение-разметка в {dataset_path}")
            return [], []
            
        logger.info(f"🔍 Обработка {len(image_label_pairs)} пар изображение-разметка для получения масок...")
        
        for i, (img_path, label_path) in enumerate(image_label_pairs):
            if i % 50 == 0:
                logger.info(f"   Обработано {i}/{len(image_label_pairs)} изображений (ошибок: {error_count})")
            
            try:
                # Загрузка изображения
                try:
                    image = Image.open(img_path)
                    img_width, img_height = image.size
                    
                    if img_width <= 0 or img_height <= 0:
                        logger.warning(f"⚠️ Некорректные размеры изображения {img_path}: {img_width}x{img_height}")
                        error_count += 1
                        continue
                        
                except Exception as e:
                    logger.warning(f"⚠️ Ошибка загрузки изображения {img_path}: {e}")
                    error_count += 1
                    continue
                
                # Получение предсказания модели
                try:
                    results = model.predict(img_path, imgsz=img_size, conf=conf_threshold, verbose=False)
                    if results and len(results) > 0:
                        pred_mask = create_mask_from_yolo_results(results[0], img_width, img_height)
                    else:
                        logger.debug(f"🔍 Пустые результаты предсказания для {img_path}")
                        pred_mask = np.zeros((img_height, img_width), dtype=np.uint8)
                        
                    if pred_mask is None:
                        logger.warning(f"⚠️ create_mask_from_yolo_results вернула None для {img_path}")
                        pred_mask = np.zeros((img_height, img_width), dtype=np.uint8)
                        
                except Exception as e:
                    logger.warning(f"⚠️ Ошибка предсказания для {img_path}: {e}")
                    pred_mask = np.zeros((img_height, img_width), dtype=np.uint8)
                    error_count += 1
                
                # Загрузка истинной разметки
                try:
                    if os.path.exists(label_path):
                        gt_mask = create_mask_from_yolo_labels(label_path, img_width, img_height)
                        
                        if gt_mask is None:
                            logger.warning(f"⚠️ create_mask_from_yolo_labels вернула None для {label_path}")
                            gt_mask = np.zeros((img_height, img_width), dtype=np.uint8)
                    else:
                        logger.debug(f"🔍 Файл разметки не найден: {label_path}")
                        gt_mask = np.zeros((img_height, img_width), dtype=np.uint8)
                        
                except Exception as e:
                    logger.warning(f"⚠️ Ошибка загрузки разметки для {label_path}: {e}")
                    gt_mask = np.zeros((img_height, img_width), dtype=np.uint8)
                    error_count += 1
                
                # Проверка размеров масок
                if pred_mask.shape != gt_mask.shape:
                    logger.debug(f"🔧 Несоответствие размеров масок: pred={pred_mask.shape}, gt={gt_mask.shape}")
                
                # Изменение размера масок до стандартного размера
                try:
                    if pred_mask.shape != (img_size, img_size):
                        pred_mask = cv2.resize(pred_mask, (img_size, img_size), interpolation=cv2.INTER_NEAREST)
                    if gt_mask.shape != (img_size, img_size):
                        gt_mask = cv2.resize(gt_mask, (img_size, img_size), interpolation=cv2.INTER_NEAREST)
                except Exception as e:
                    logger.warning(f"⚠️ Ошибка изменения размера масок для {img_path}: {e}")
                    pred_mask = np.zeros((img_size, img_size), dtype=np.uint8)
                    gt_mask = np.zeros((img_size, img_size), dtype=np.uint8)
                    error_count += 1
                
                predictions.append(pred_mask)
                ground_truths.append(gt_mask)
                processed_count += 1
                
            except Exception as e:
                logger.error(f"❌ Критическая ошибка обработки пары {i+1}/{len(image_label_pairs)} ({img_path}): {e}")
                error_count += 1
                continue
            
    else:  # Плоская структура
        logger.info(f"📄 Обнаружена плоская структура датасета")
        
        try:
            image_files = [f for f in os.listdir(images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        except Exception as e:
            logger.error(f"❌ Ошибка чтения файлов изображений: {e}")
            return [], []
        
        logger.info(f"🔍 Найдено {len(image_files)} изображений в плоской структуре")
        logger.info(f"🔍 Обработка {len(image_files)} изображений для получения масок...")
        
        for i, img_file in enumerate(image_files):
            if i % 50 == 0:
                logger.info(f"   Обработано {i}/{len(image_files)} изображений (ошибок: {error_count})")
                
            img_path = os.path.join(images_dir, img_file)
            label_path = os.path.join(labels_dir, img_file.rsplit('.', 1)[0] + '.txt')
            
            try:
                # Загрузка изображения
                try:
                    image = Image.open(img_path)
                    img_width, img_height = image.size
                    
                    if img_width <= 0 or img_height <= 0:
                        logger.warning(f"⚠️ Некорректные размеры изображения {img_file}: {img_width}x{img_height}")
                        error_count += 1
                        continue
                        
                except Exception as e:
                    logger.warning(f"⚠️ Ошибка загрузки изображения {img_file}: {e}")
                    error_count += 1
                    continue
                
                # Получение предсказания модели
                try:
                    results = model.predict(img_path, imgsz=img_size, conf=conf_threshold, verbose=False)
                    if results and len(results) > 0:
                        pred_mask = create_mask_from_yolo_results(results[0], img_width, img_height)
                    else:
                        logger.debug(f"🔍 Пустые результаты предсказания для {img_file}")
                        pred_mask = np.zeros((img_height, img_width), dtype=np.uint8)
                        
                    if pred_mask is None:
                        logger.warning(f"⚠️ create_mask_from_yolo_results вернула None для {img_file}")
                        pred_mask = np.zeros((img_height, img_width), dtype=np.uint8)
                        
                except Exception as e:
                    logger.warning(f"⚠️ Ошибка предсказания для {img_file}: {e}")
                    pred_mask = np.zeros((img_height, img_width), dtype=np.uint8)
                    error_count += 1
                
                # Загрузка истинной разметки
                try:
                    if os.path.exists(label_path):
                        gt_mask = create_mask_from_yolo_labels(label_path, img_width, img_height)
                        
                        if gt_mask is None:
                            logger.warning(f"⚠️ create_mask_from_yolo_labels вернула None для {label_path}")
                            gt_mask = np.zeros((img_height, img_width), dtype=np.uint8)
                    else:
                        logger.debug(f"🔍 Файл разметки не найден: {label_path}")
                        gt_mask = np.zeros((img_height, img_width), dtype=np.uint8)
                        
                except Exception as e:
                    logger.warning(f"⚠️ Ошибка загрузки разметки для {img_file}: {e}")
                    gt_mask = np.zeros((img_height, img_width), dtype=np.uint8)
                    error_count += 1
                
                # Проверка размеров масок
                if pred_mask.shape != gt_mask.shape:
                    logger.debug(f"🔧 Несоответствие размеров масок: pred={pred_mask.shape}, gt={gt_mask.shape}")
                
                # Изменение размера масок до стандартного размера
                try:
                    if pred_mask.shape != (img_size, img_size):
                        pred_mask = cv2.resize(pred_mask, (img_size, img_size), interpolation=cv2.INTER_NEAREST)
                    if gt_mask.shape != (img_size, img_size):
                        gt_mask = cv2.resize(gt_mask, (img_size, img_size), interpolation=cv2.INTER_NEAREST)
                except Exception as e:
                    logger.warning(f"⚠️ Ошибка изменения размера масок для {img_file}: {e}")
                    pred_mask = np.zeros((img_size, img_size), dtype=np.uint8)
                    gt_mask = np.zeros((img_size, img_size), dtype=np.uint8)
                    error_count += 1
                
                predictions.append(pred_mask)
                ground_truths.append(gt_mask)
                processed_count += 1
                
            except Exception as e:
                logger.error(f"❌ Критическая ошибка обработки изображения {i+1}/{len(image_files)} ({img_file}): {e}")
                error_count += 1
                continue
    
    # Финальная статистика
    logger.info(f"✅ Завершена обработка датасета {dataset_path}")
    logger.info(f"📊 Статистика: обработано {processed_count} изображений, ошибок {error_count}")
    logger.info(f"📊 Получено {len(predictions)} масок предсказаний и {len(ground_truths)} GT масок")
    
    # Проверка на пустые результаты
    if not predictions or not ground_truths:
        logger.error(f"❌ Получены пустые списки масок! predictions: {len(predictions)}, ground_truths: {len(ground_truths)}")
        return [], []
    
    # Проверка соответствия количества
    if len(predictions) != len(ground_truths):
        logger.error(f"❌ Несоответствие количества масок! predictions: {len(predictions)}, ground_truths: {len(ground_truths)}")
        return [], []
    
    return predictions, ground_truths


def create_mask_from_yolo_results(results, img_width: int, img_height: int) -> np.ndarray:
    """
    Создание бинарной маски из результатов YOLO
    
    Args:
        results: Результаты детекции YOLO
        img_width: Ширина изображения
        img_height: Высота изображения
        
    Returns:
        np.ndarray: Бинарная маска
    """
    mask = np.zeros((img_height, img_width), dtype=np.uint8)
    
    if results.boxes is not None and len(results.boxes) > 0:
        boxes = results.boxes.xyxy.cpu().numpy()  # Координаты bbox в формате x1,y1,x2,y2
        
        for box in boxes:
            x1, y1, x2, y2 = map(int, box[:4])
            # Ограничиваем координаты размерами изображения
            x1 = max(0, min(x1, img_width-1))
            y1 = max(0, min(y1, img_height-1))
            x2 = max(0, min(x2, img_width-1))
            y2 = max(0, min(y2, img_height-1))
            
            # Заполняем область bbox единицами
            mask[y1:y2+1, x1:x2+1] = 1
    
    return mask


def create_mask_from_yolo_labels(label_path: str, img_width: int, img_height: int) -> np.ndarray:
    """
    Создание бинарной маски из YOLO разметки
    
    Args:
        label_path: Путь к файлу разметки
        img_width: Ширина изображения
        img_height: Высота изображения
        
    Returns:
        np.ndarray: Бинарная маска
    """
    mask = np.zeros((img_height, img_width), dtype=np.uint8)
    
    if not os.path.exists(label_path):
        return mask
    
    try:
        with open(label_path, 'r') as f:
            lines = f.readlines()
        
        for line in lines:
            parts = line.strip().split()
            if len(parts) >= 5:
                # YOLO формат: class_id center_x center_y width height (нормализованные координаты)
                center_x = float(parts[1]) * img_width
                center_y = float(parts[2]) * img_height
                width = float(parts[3]) * img_width
                height = float(parts[4]) * img_height
                
                # Преобразование в координаты bbox
                x1 = int(center_x - width/2)
                y1 = int(center_y - height/2)
                x2 = int(center_x + width/2)
                y2 = int(center_y + height/2)
                
                # Ограничиваем координаты размерами изображения
                x1 = max(0, min(x1, img_width-1))
                y1 = max(0, min(y1, img_height-1))
                x2 = max(0, min(x2, img_width-1))
                y2 = max(0, min(y2, img_height-1))
                
                # Заполняем область bbox единицами
                mask[y1:y2+1, x1:x2+1] = 1
    
    except Exception as e:
        logger.warning(f"⚠️ Ошибка чтения разметки {label_path}: {e}")
    
    return mask


def calculate_iou(region_a: np.ndarray, region_b: np.ndarray) -> float:
    """
    Вычисление IoU между двумя областями по формуле (3)
    
    Args:
        region_a: Первая область (бинарная маска)
        region_b: Вторая область (бинарная маска)
        
    Returns:
        float: Значение IoU
    """
    intersection = np.logical_and(region_a, region_b).sum()
    union = np.logical_or(region_a, region_b).sum()
    
    if union == 0:
        return 0.0
    
    return intersection / union


def create_iou_matrix(pred_regions: List[np.ndarray], gt_regions: List[np.ndarray]) -> np.ndarray:
    """
    Создание матрицы IoU между предсказанными и истинными областями
    
    Args:
        pred_regions: Список предсказанных областей
        gt_regions: Список истинных областей
        
    Returns:
        np.ndarray: Матрица IoU размером [len(pred_regions), len(gt_regions)]
    """
    iou_matrix = np.zeros((len(pred_regions), len(gt_regions)))
    
    for i, pred_region in enumerate(pred_regions):
        for j, gt_region in enumerate(gt_regions):
            iou_matrix[i, j] = calculate_iou(pred_region, gt_region)
    
    return iou_matrix


def count_tp_fp_fn_from_matrix(iou_matrix: np.ndarray, threshold: float) -> Tuple[int, int, int]:
    """
    Подсчет TP, FP, FN из матрицы IoU согласно описанному алгоритму
    
    Args:
        iou_matrix: Матрица IoU
        threshold: Пороговое значение
        
    Returns:
        Tuple[int, int, int]: TP, FP, FN
    """
    tp = 0
    matrix_copy = iou_matrix.copy()
    
    while matrix_copy.size > 0:
        # Поиск максимального элемента
        max_val = np.max(matrix_copy)
        
        if max_val >= threshold:
            # Увеличиваем TP
            tp += 1
            
            # Находим позицию максимального элемента
            max_pos = np.unravel_index(np.argmax(matrix_copy), matrix_copy.shape)
            row_idx, col_idx = max_pos
            
            # Удаляем строку и столбец
            matrix_copy = np.delete(matrix_copy, row_idx, axis=0)
            matrix_copy = np.delete(matrix_copy, col_idx, axis=1)
        else:
            # Прерываем процедуру
            break
    
    # FN = количество оставшихся столбцов (истинных объектов)
    fn = matrix_copy.shape[1] if matrix_copy.size > 0 else 0
    
    # FP = количество оставшихся строк (предсказанных объектов)
    fp = matrix_copy.shape[0] if matrix_copy.size > 0 else 0
    
    return tp, fp, fn


def cleanup_debug_files():
    """Очистка временных debug файлов"""
    debug_base_dir = '/kaggle/working'
    
    if os.path.exists(debug_base_dir):
        try:
            shutil.rmtree(debug_base_dir)
            logger.info("🧹 Временные debug файлы очищены")
        except Exception as e:
            logger.warning(f"⚠️ Ошибка очистки debug файлов: {e}")
    
    # Очистка временных YAML файлов
    temp_yaml_files = ['kaggle_combined_data.yaml', 'combined_data.yaml', 'private_validation.yaml']
    for yaml_file in temp_yaml_files:
        if os.path.exists(yaml_file):
            try:
                os.remove(yaml_file)
            except:
                pass

def analyze_all_datasets() -> Dict:
    """Анализ всех доступных датасетов"""
    logger.info("📊 Анализ всех датасетов...")
    
    datasets = {
        'train_dataset_1': get_dataset_statistics(TRAIN_DATASET_1),
        'train_dataset_2': get_dataset_statistics(TRAIN_DATASET_2),
        'val_dataset_public': get_dataset_statistics(VAL_DATASET_PUBLIC),
        'val_dataset_private': get_dataset_statistics(VAL_DATASET_PRIVATE)
    }
    
    total_train_images = 0
    total_train_labels = 0
    
    for name, stats in datasets.items():
        if stats['exists']:
            structure_info = f" ({stats['structure_type']})" if 'structure_type' in stats else ""
            logger.info(f"✅ {name}{structure_info}: {stats['images_count']} изображений, {stats['labels_count']} меток")
            if 'train' in name:
                total_train_images += stats['images_count']
                total_train_labels += stats['labels_count']
            if stats['classes_distribution']:
                logger.info(f"   Распределение классов: {stats['classes_distribution']}")
        else:
            logger.warning(f"❌ {name}: датасет не найден")
    
    logger.info(f"🎯 Итого для обучения: {total_train_images} изображений, {total_train_labels} меток")
    
    return datasets

# ===== ФУНКЦИИ ИЗ METRIC.PY =====

# Константы для метрик
PUBLIC_GT_CSV_PATH: str = 'public_gt_solution_24-10-24.csv'
COLUMNS = ['image_id', 'label', 'xc', 'yc', 'w', 'h', 'w_img', 'h_img', 'score', 'time_spent']
EXTENSION = '.jpg'  # Константа из metric_counter.py

def df_to_bytes(df: pd.DataFrame) -> bytes:
    df_byte: bytes = df.to_json().encode(encoding="utf-8")
    
    return df_byte

def bytes_to_dict(df_byte: bytes) -> dict:
    if isinstance(df_byte, bytes):
        df_byte = df_byte.decode("utf-8")
    df_byte = df_byte.replace("'", '"')
    df_dict: dict = json.loads(df_byte)
    
    return df_dict

def bytes_to_df(df_byte: bytes) -> pd.DataFrame:
    predicted_dict = bytes_to_dict(df_byte)
    predicted_df = pd.DataFrame(predicted_dict)
    
    return predicted_df

def open_df_as_bytes(csv_path: str) -> bytes:
    df = pd.read_csv(csv_path, sep=",", decimal=".", 
                     converters={'image_id': str,
                                 'time_spent': float})
    df_bytes = df_to_bytes(df)

    return df_bytes

def set_types(df: pd.DataFrame) -> pd.DataFrame:
    return df.astype({'image_id': str,
        'label': int,
        'xc': float,
        'yc': float,
        'w': float,
        'h': float,
        'w_img': int,
        'h_img': int,
        },
        errors = 'ignore'
    )

def get_time_spent(df: pd.DataFrame, m: int) -> np.ndarray:
    # Проверяем, что time_spent для каждого image_id одинаковые
    for image_id, group in df.groupby('image_id'):
        assert group['time_spent'].nunique() == 1, f"Разные значения time_spent для image_id: {image_id}"
    
    # Получаем первое значение time_spent для каждого уникального image_id
    time_spent = df.groupby('image_id')['time_spent'].first()
    time_spent = time_spent.reset_index()['time_spent'].values
    assert len(time_spent) == m, f'Количество объектов time_spent должно быть {m} (у вас {len(time_spent)})'
    
    return time_spent

def preprocess_predicted_df(predicted_file: bytes, gt_file: bytes, m: int) -> Tuple[pd.DataFrame, pd.DataFrame, np.ndarray]:
    # Преобразуем байткод в датафрейм
    if gt_file is None:
        gt_file: bytes = open_df_as_bytes(PUBLIC_GT_CSV_PATH)
    predicted_df = bytes_to_df(predicted_file)
    gt_df = bytes_to_df(gt_file)
    # Валидируем данные
    assert set(predicted_df.columns) == set(COLUMNS), Exception(f'Ошибка названия столбцов в датафрейме: у вас {list(predicted_df.columns)}, должны быть {COLUMNS}')

    assert 'score' in predicted_df, "Датафрейм должен содержать столбец score времени инференса на изображении"
    assert not np.any(predicted_df['w'].values < 0.0) and not np.any(predicted_df['w'].values > 1), "Ширина (w) должна быть в пределах [0, 1]."
    assert not np.any(predicted_df['h'].values < 0.0) and not np.any(predicted_df['h'].values > 1), "Высота (h) должна быть в пределах [0, 1]."
    assert not np.any(predicted_df['w_img'].values < 1) and not np.any(predicted_df['w_img'].values > 15000), "Ширина (w_img) должна быть в пределах [0, 15000]."
    assert not np.any(predicted_df['h_img'].values < 1) and not np.any(predicted_df['h_img'].values > 15000), "Высота (h_img) должна быть в пределах [0, 15000]."
    assert not np.any(predicted_df['xc'].values < 0.0) and not np.any(predicted_df['xc'].values > 1.0), "Центр объекта (xc) должен быть в пределах [0, 1]."
    assert not np.any(predicted_df['yc'].values < 0.0) and not np.any(predicted_df['yc'].values > 1.0), "Центр объекта (yc) должен быть в пределах [0, 1]."
    assert not np.any(predicted_df['score'].values < 0.0) and not np.any(predicted_df['score'].values > 1.0), "Столбец score должен быть в пределах [0, 1]"
    assert 'time_spent' in predicted_df, "Датафрейм должен содержать столбец time_spent времени инференса на изображении"
    
    # Забираем время, в том числе для пустых предсказаний, и удаляем из df
    time_spent = get_time_spent(df=predicted_df, m=m)
    del predicted_df['time_spent']
    predicted_df = predicted_df.dropna()
    
    # Приводим форматы столбцов к типам
    predicted_df = set_types(predicted_df)
    gt_df = set_types(gt_df)
    
    # Делаем image_id индексом и сортируем, чтобы сохранить порядок индексов
    gt_df = gt_df.set_index('image_id').sort_index()
    predicted_df = predicted_df.set_index('image_id').sort_index()
    
    # Получаем все уникальные индексы изображений
    unique_image_ids = tuple(set(predicted_df.index.to_list() + gt_df.index.to_list()))
    assert len(unique_image_ids) <= m, Exception(f"Количество уникальных ID изображений не должно превышать {m}!")
    assert not predicted_df.empty and not gt_df.empty, "Датафреймы не должны быть пустыми"

    return gt_df, predicted_df, time_spent


def get_box_coordinates(row):
    """Преобразует центр и размеры в координаты углов бокса"""
    w_img = int(row['w_img'])
    h_img = int(row['h_img'])
    
    x1 = int((row['xc'] - row['w']/2) * w_img)
    y1 = int((row['yc'] - row['h']/2) * h_img)
    x2 = int((row['xc'] + row['w']/2) * w_img)
    y2 = int((row['yc'] + row['h']/2) * h_img)
    
    return (x1, y1, x2, y2)


@jit(nopython=True)
def compute_iou_from_coords(pred_box, gt_box):
    """Вычисляет IoU между двумя боксами по координатам"""
    # pred_box и gt_box в формате (x1, y1, x2, y2)
    x1_p, y1_p, x2_p, y2_p = pred_box
    x1_g, y1_g, x2_g, y2_g = gt_box
    # Находим координаты пересечения
    x_left = max(x1_p, x1_g)
    y_top = max(y1_p, y1_g)
    x_right = min(x2_p, x2_g)
    y_bottom = min(y2_p, y2_g)
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    
    # Площади боксов
    box1_area = (x2_p - x1_p) * (y2_p - y1_p)
    box2_area = (x2_g - x1_g) * (y2_g - y1_g)
    union_area = box1_area + box2_area - intersection_area
    
    return intersection_area / union_area if union_area > 0 else 0.0


def process_image(pred_df, gt_df, thresholds):
    """Обработка одного изображения"""
    pred_boxes = [get_box_coordinates(row) for _, row in pred_df.iterrows()]
    gt_boxes = [get_box_coordinates(row) for _, row in gt_df.iterrows()]
    
    num_pred = len(pred_boxes)
    num_gt = len(gt_boxes)
    iou_matrix = np.zeros((num_pred, num_gt))
    
    for i, pred_box in enumerate(pred_boxes):
        for j, gt_box in enumerate(gt_boxes):
            iou_matrix[i, j] = compute_iou_from_coords(pred_box, gt_box)
    
    results = {}
    for t in thresholds:
        matches = []
        iou_mat = iou_matrix.copy()
        iou_mat[iou_mat < t] = 0
        
        pred_indices = set()
        gt_indices = set()
        
        while True:
            max_iou = iou_mat.max()
            if max_iou == 0:
                break
            i, j = np.unravel_index(np.argmax(iou_mat), iou_mat.shape)
            if i not in pred_indices and j not in gt_indices:
                pred_indices.add(i)
                gt_indices.add(j)
                matches.append((i, j))
            iou_mat[i, :] = 0
            iou_mat[:, j] = 0
        
        tp = len(matches)
        fp = num_pred - tp
        fn = num_gt - tp
        
        results[t] = {'tp': tp, 'fp': fp, 'fn': fn}
    
    return results


def compute_overall_metric(
        predicted_df: pd.DataFrame,
        gt_df: pd.DataFrame,
        time_spent: np.ndarray,
        thresholds: np.ndarray,
        beta: float,
        m: int,
        parallelize: bool = True
    ) -> np.float64:
    # Получаем все уникальные индексы изображений
    unique_image_ids = tuple(set(predicted_df.index.to_list() + gt_df.index.to_list()))

    total_tp = {t: 0 for t in thresholds}
    total_fp = {t: 0 for t in thresholds}
    total_fn = {t: 0 for t in thresholds}
    
    def process_image_id(image_id):
        pred_df_image_id = predicted_df[predicted_df.index == image_id]
        gt_df_image_id = gt_df[gt_df.index == image_id]
        
        # Случай, когда истинные значения есть, а предсказаний нет
        if not gt_df_image_id.empty and pred_df_image_id.empty:
            num_gt = np.float64(len(gt_df_image_id))
            return {t: {'tp': 0, 'fp': 0, 'fn': num_gt} for t in thresholds}
        
        # Случай, когда предсказания есть, а истинных значений нет
        elif not pred_df_image_id.empty and gt_df_image_id.empty:
            num_pred = np.float64(len(pred_df_image_id))
            return {t: {'tp': 0, 'fp': num_pred, 'fn': 0} for t in thresholds}
        
        # Оба случая не пустые
        elif not pred_df_image_id.empty and not gt_df_image_id.empty:
            # Вместо работы с масками передаем напрямую датафреймы
            result = process_image(pred_df_image_id, gt_df_image_id, thresholds)
            return result
        
        return None

    results = []
    if parallelize:
        # Распараллеленное многопоточное выполнение
        with ThreadPoolExecutor() as executor:
            results = list(executor.map(process_image_id, unique_image_ids))
    else:
        # Последовательный цикл выполнения
        for image_id in unique_image_ids:
            results.append(process_image_id(image_id))

    for result in results:
        if result:
            for t in thresholds:
                total_tp[t] += result[t]['tp']
                total_fp[t] += result[t]['fp']
                total_fn[t] += result[t]['fn']

    metric, accuracy, fp_rate, avg_time = metric_counter(
        time_spent=time_spent,
        total_tp=total_tp,
        total_fp=total_fp,
        total_fn=total_fn,
        thresholds=thresholds,
        beta=beta,
        m=m)
    
    return metric, accuracy, fp_rate, avg_time


def metric_counter(
        time_spent: np.ndarray,
        total_tp: dict,
        total_fp: dict,
        total_fn: dict,
        thresholds: np.ndarray,
        beta: float,
        m: int) -> np.float64:
    # Рачет Q (F-beta score)
    f_beta_scores = [] 
    beta_squared = beta ** 2
    for t in thresholds:
        tp = total_tp[t]
        fp = total_fp[t]
        fn = total_fn[t]
        numerator = (1 + beta_squared) * tp
        denominator = (1 + beta_squared) * tp + beta_squared * fn + fp
        if denominator == 0:
            f_beta_t = 0.0
        else:
            f_beta_t = numerator / denominator
        f_beta_scores.append(f_beta_t)
    
    # Метрика теперь равна только F-beta score (без учета скорости)
    metric = (1 / len(thresholds)) * np.sum(f_beta_scores)
    metric = np.round(metric, decimals=10)
    
    # Точность обнаружения
    accuracy = {float(t): round(float((total_tp[t] / (total_tp[t] + total_fp[t])) * 100), 3) \
            if (total_tp[t] + total_fp[t]) > 0 else 0 for t in thresholds}
    # Число ложноположительных срабатываний 
    fp_rate = {float(t): round(float((total_fp[t] / (total_fp[t] + total_tp[t])) * 100), 3) \
            if (total_fp[t] + total_tp[t]) > 0 else 0 for t in thresholds}
    # Среднее время на обработку снимка 
    avg_time = round(float(np.mean(time_spent)), 3)

    return metric, accuracy, fp_rate, avg_time


def evaluate(predicted_file: bytes,
        gt_file: bytes = None,
        thresholds: np.ndarray = np.round(np.arange(0.3, 1.0, 0.07), 2),
        beta: float = 1.0,
        m: int = 500,
        parallelize: bool = True) -> float:
    metric, accuracy, fp_rate, avg_time = 0.0, {}, {}, 0.0
    try:
        # Валидация данных, конвертация датафреймов, приведение типов
        gt_df, predicted_df, time_spent = preprocess_predicted_df(gt_file=gt_file,
                                                predicted_file=predicted_file,
                                                m=m
        )
        # Расчет метрики, выполняется параллельно (parallelize=True)
        metric, accuracy, fp_rate, avg_time = compute_overall_metric(predicted_df=predicted_df,
                    gt_df=gt_df,
                    thresholds=thresholds,
                    beta=beta,
                    m=m,
                    parallelize=parallelize,
                    time_spent=time_spent
        )
    except Exception as e:
        raise Exception(f"Произошла ошибка выполнения скрипта: {str(e)}")
    
    return metric, accuracy, fp_rate, avg_time


def predict(images: List[np.ndarray]) -> List[List[Dict]]:
    """Функция предсказания для обученной YOLO модели
    
    Args:
        images: Список изображений в формате numpy array (RGB)
        
    Returns:
        List[List[Dict]]: Список результатов для каждого изображения,
                         где каждый результат содержит словари с ключами:
                         'xc', 'yc', 'w', 'h', 'label', 'score'
    """
    try:
        # Загружаем лучшую обученную модель
        model_path = './models/yolov13_uav_runs/dynamic_routing_experiment/weights/best.pt'
        if not os.path.exists(model_path):
            # Попробуем альтернативные пути
            alternative_paths = [
                './yolov13_uav_runs/dynamic_routing_experiment/weights/best.pt',
                './runs/detect/train/weights/best.pt',
                './best.pt'
            ]
            for alt_path in alternative_paths:
                if os.path.exists(alt_path):
                    model_path = alt_path
                    break
            else:
                logger.error(f"Модель не найдена по пути {model_path}")
                return [[] for _ in images]
        
        # Загружаем модель
        model = YOLO(model_path)
        logger.info(f"Модель загружена из {model_path}")
        
        all_results = []
        
        for image in images:
            image_results = []
            
            # Конвертируем numpy array в PIL Image для YOLO
            if isinstance(image, np.ndarray):
                # Убеждаемся что изображение в правильном формате
                if image.dtype != np.uint8:
                    image = (image * 255).astype(np.uint8)
                
                # YOLO ожидает BGR, но у нас RGB
                image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            else:
                image_bgr = image
            
            # Получаем размеры изображения
            h_img, w_img = image_bgr.shape[:2]
            
            # Выполняем предсказание
            results = model(image_bgr, conf=0.25, iou=0.45, verbose=False)
            
            # Обрабатываем результаты
            if results and len(results) > 0:
                result = results[0]
                if result.boxes is not None and len(result.boxes) > 0:
                    boxes = result.boxes
                    
                    for i in range(len(boxes)):
                        # Получаем координаты в формате xyxy
                        xyxy = boxes.xyxy[i].cpu().numpy()
                        conf = float(boxes.conf[i].cpu().numpy())
                        cls = int(boxes.cls[i].cpu().numpy())
                        
                        # Конвертируем xyxy в xywh (центр + размеры)
                        x1, y1, x2, y2 = xyxy
                        xc = (x1 + x2) / 2
                        yc = (y1 + y2) / 2
                        w = x2 - x1
                        h = y2 - y1
                        
                        # Нормализуем координаты относительно размера изображения
                        xc_norm = xc / w_img
                        yc_norm = yc / h_img
                        w_norm = w / w_img
                        h_norm = h / h_img
                        
                        detection = {
                            'xc': float(xc_norm),
                            'yc': float(yc_norm),
                            'w': float(w_norm),
                            'h': float(h_norm),
                            'label': cls,
                            'score': conf
                        }
                        image_results.append(detection)
            
            all_results.append(image_results)
        
        return all_results
        
    except Exception as e:
        logger.error(f"Ошибка в функции predict: {str(e)}")
        return [[] for _ in images]


def process_images_adapted(images_path: str, result_csv_path: str = None) -> pd.DataFrame:
    """Адаптированная функция обработки папки с изображениями из metric_counter.py
    
    Args:
        images_path (str): путь до директории с изображениями
        result_csv_path (str, optional): Путь, куда сохранять результат предсказаний

    Returns:
        pd.DataFrame: датафрейм с результатом предсказаний
    """
    # Константы из metric_counter.py
    EXTENSION = '.jpg'
    COLUMNS = ['image_id', 'label', 'xc', 'yc', 'w', 'h', 'w_img', 'h_img', 'score', 'time_spent']
    
    # Фиксируем сиды
    SEED = 42
    random.seed(SEED)
    np.random.seed(SEED)
    
    # Тестовая папка должна содержать подпапку images
    images_dir = os.path.join(images_path, 'images')
    image_paths = list(Path(images_dir).glob(f'*{EXTENSION}'))
    # Перемешиваем пути c изображениями
    random.shuffle(image_paths)
    images_count = len(os.listdir(images_dir))
    
    # Проверяем, существует ли директория, изображения
    if not os.path.exists(images_dir):
        raise Exception(f"Директория {images_dir} не найдена!")
    if images_count == 0:
        raise Exception(f'Отсутствуют изображения в папке {images_dir}')
    
    results = []
    # Обрабатываем изображения по одному
    for image_path in image_paths:
        image_id = os.path.basename(image_path).split(EXTENSION)[0]
        # Открываем изображение в RGB формате
        image = cv2.imread(str(image_path), -1)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h_img, w_img, _ = image.shape

        # Засекаем время выполнения функции predict
        start_time = time.time()
        # Вызываем функцию predict для одного изображения
        image_results = predict([image])
        # Останавливаем таймер
        elapsed_time = time.time() - start_time
        time_per_image = round(elapsed_time, 4)
        
        # Дополняем результаты ID изображения и затраченным временем
        if image_results and image_results[0]:
            for res in image_results[0]:
                res['image_id'] = image_id
                res['time_spent'] = time_per_image
                res['w_img'] = w_img
                res['h_img'] = h_img
                results.append(res)
        else:
            res = {'xc': None,
                   'yc': None,
                   'w': None,
                   'h': None,
                   'label': 0,
                   'score': None,
                   'image_id': image_id,
                   'time_spent': time_per_image,
                   'w_img': None,
                   'h_img': None
            }
            results.append(res)

    result_df = pd.DataFrame()
    if results and result_csv_path:
        result_df = pd.DataFrame(results, columns=COLUMNS)
        result_df = result_df.fillna(value=np.nan)
        result_df.to_csv(result_csv_path, index=False, na_rep=np.nan)
    
    logger.info('Обработка выборки выполнена успешно!')
    
    return result_df


class AdvancedDynamicRoutingModule(nn.Module):
    """Продвинутый модуль динамической маршрутизации с многоуровневой адаптацией для YOLOv13"""
    
    def __init__(self, in_channels: int, out_channels: int, num_routes: int = 3, 
                 routing_iterations: int = 2, capsule_dim: int = 8):
        super().__init__()
        self.num_routes = num_routes
        self.routing_iterations = routing_iterations
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.capsule_dim = capsule_dim
        
        # Упрощенные капсулы для эффективности
        self.primary_capsules = nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(in_channels, out_channels // num_routes, 3, padding=1),
                nn.BatchNorm2d(out_channels // num_routes),
                nn.SiLU()
            ) for _ in range(num_routes)
        ])
        
        # Адаптивные весовые матрицы
        self.routing_weights = nn.Parameter(torch.randn(num_routes, capsule_dim, capsule_dim) * 0.1)
        self.temperature = nn.Parameter(torch.ones(1))
        
        # UAV-специфичные адаптации
        self.altitude_encoder = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(out_channels // num_routes, capsule_dim, 1),
            nn.SiLU()
        )
        
        # Объединение маршрутов
        self.route_fusion = nn.Conv2d(out_channels, out_channels, 1)
        self.norm = nn.BatchNorm2d(out_channels)
        
    def enhanced_squash(self, tensor: torch.Tensor, dim: int = -1, epsilon: float = 1e-8) -> torch.Tensor:
        """Улучшенная функция сжатия"""
        squared_norm = (tensor ** 2).sum(dim=dim, keepdim=True)
        scale = squared_norm / (1 + squared_norm + epsilon)
        unit_vector = tensor / torch.sqrt(squared_norm + epsilon)
        return scale * unit_vector
    
    def dynamic_routing(self, route_features: List[torch.Tensor]) -> torch.Tensor:
        """Динамическая маршрутизация между путями"""
        batch_size = route_features[0].size(0)
        device = route_features[0].device
        
        # Инициализация весов маршрутизации
        routing_logits = torch.zeros(batch_size, self.num_routes, device=device)
        
        for iteration in range(self.routing_iterations):
            # Softmax для получения весов
            routing_weights = F.softmax(routing_logits / self.temperature, dim=1)
            
            # Взвешенное объединение
            weighted_features = []
            for i, features in enumerate(route_features):
                weight = routing_weights[:, i:i+1, None, None]
                weighted_features.append(features * weight)
            
            combined = torch.stack(weighted_features, dim=1).sum(dim=1)
            
            if iteration < self.routing_iterations - 1:
                # Обновление логитов на основе согласованности
                for i, features in enumerate(route_features):
                    agreement = F.cosine_similarity(
                        features.flatten(1), combined.flatten(1), dim=1
                    )
                    routing_logits[:, i] += agreement
        
        return combined
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Обработка через различные маршруты
        route_outputs = []
        for capsule in self.primary_capsules:
            route_out = capsule(x)
            route_outputs.append(route_out)
        
        # Динамическая маршрутизация
        routed_output = self.dynamic_routing(route_outputs)
        
        # Объединение всех маршрутов
        all_routes = torch.cat(route_outputs, dim=1)
        fused = self.route_fusion(all_routes)
        
        # Остаточное соединение с маршрутизированным выходом
        if routed_output.size(1) == fused.size(1):
            output = fused + 0.3 * routed_output
        else:
            output = fused
        
        return self.norm(output)

class EnhancedUAVAdaptiveBlock(nn.Module):
    """Улучшенный адаптивный блок для UAV с многофакторной адаптацией"""
    
    def __init__(self, in_channels: int, out_channels: int, reduction_ratio: int = 16):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        
        # Основные конволюционные слои
        self.conv1 = Conv(in_channels, out_channels, 3, 1)
        self.conv2 = Conv(out_channels, out_channels, 3, 1)
        
        # Канальное внимание (упрощенное)
        self.channel_attention = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(out_channels, max(out_channels // reduction_ratio, 1), 1),
            nn.SiLU(),
            nn.Conv2d(max(out_channels // reduction_ratio, 1), out_channels, 1),
            nn.Sigmoid()
        )
        
        # Пространственное внимание
        self.spatial_attention = nn.Sequential(
            nn.Conv2d(2, 1, 7, padding=3),
            nn.Sigmoid()
        )
        
        # UAV-специфичные адаптации
        self.altitude_adaptation = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(out_channels, out_channels, 1),
            nn.Sigmoid()
        )
        
        # Проекция для остаточного соединения
        self.shortcut = nn.Identity() if in_channels == out_channels else Conv(in_channels, out_channels, 1, 1)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        residual = self.shortcut(x)
        
        # Основная обработка
        out = self.conv1(x)
        out = self.conv2(out)
        
        # Канальное внимание
        ca = self.channel_attention(out)
        out = out * ca
        
        # Пространственное внимание
        avg_pool = torch.mean(out, dim=1, keepdim=True)
        max_pool, _ = torch.max(out, dim=1, keepdim=True)
        spatial_input = torch.cat([avg_pool, max_pool], dim=1)
        sa = self.spatial_attention(spatial_input)
        out = out * sa
        
        # UAV адаптация
        altitude_att = self.altitude_adaptation(out)
        out = out * altitude_att
        
        # Остаточное соединение
        return out + residual

class YOLOv13DynamicUAV(nn.Module):
    """YOLOv13 с динамической маршрутизацией для UAV"""
    
    def __init__(self, base_model, num_classes=1):
        super().__init__()
        self.base_model = base_model
        self.num_classes = num_classes
        
        # Интеграция динамических модулей в backbone
        self.dynamic_modules = nn.ModuleList([
            AdvancedDynamicRoutingModule(64, 64),
            AdvancedDynamicRoutingModule(128, 128),
            AdvancedDynamicRoutingModule(256, 256),
        ])
        
        # UAV адаптивные блоки
        self.uav_blocks = nn.ModuleList([
            EnhancedUAVAdaptiveBlock(64, 64),
            EnhancedUAVAdaptiveBlock(128, 128),
            EnhancedUAVAdaptiveBlock(256, 256),
        ])
        
    def forward(self, x):
        # Используем базовую YOLO модель с интеграцией наших модулей
        return self.base_model(x)

def calculate_optimal_img_size(device: str, base_size: int = 640) -> int:
    """Расчет оптимального размера изображения на основе доступной памяти GPU"""
    if not device.startswith('cuda'):
        return base_size
    
    try:
        # Получаем информацию о памяти GPU
        total_memory = torch.cuda.get_device_properties(0).total_memory
        allocated_memory = torch.cuda.memory_allocated(0)
        cached_memory = torch.cuda.memory_reserved(0)
        free_memory = total_memory - max(allocated_memory, cached_memory)
        
        # Конвертируем в GB для удобства
        total_gb = total_memory / (1024**3)
        free_gb = free_memory / (1024**3)
        
        logger.info(f"💾 Память GPU: {total_gb:.1f}GB общая, {free_gb:.1f}GB свободная")
        
        # Определяем оптимальный размер на основе доступной памяти
        if total_gb <= 8:  # Слабые GPU
            optimal_size = min(base_size, 512)
        elif total_gb <= 16:  # T4, RTX 3060 и подобные
            if free_gb >= 10:
                optimal_size = min(base_size, 832)
            elif free_gb >= 6:
                optimal_size = min(base_size, 640)
            else:
                optimal_size = min(base_size, 512)
        else:  # Мощные GPU
            optimal_size = base_size
        
        logger.info(f"🎯 Оптимальный размер изображения: {optimal_size}x{optimal_size}")
        return optimal_size
        
    except Exception as e:
        logger.warning(f"⚠️ Ошибка расчета размера изображения: {e}, используем базовый размер {base_size}")
        return base_size

def optimize_gpu_memory():
    """Оптимизация GPU памяти для Kaggle"""
    if torch.cuda.is_available():
        # Очистка кэша CUDA
        torch.cuda.empty_cache()
        
        # Принудительная сборка мусора
        gc.collect()
        
        # Очистка неиспользуемых кэшированных объектов
        if hasattr(torch.cuda, 'synchronize'):
            torch.cuda.synchronize()
        
        # Устанавливаем ограничение на использование памяти GPU (75% для безопасности)
        torch.cuda.set_per_process_memory_fraction(0.75)
        
        # Дополнительная очистка для PyTorch
        if hasattr(torch.cuda, 'ipc_collect'):
            torch.cuda.ipc_collect()
        
        # Логирование состояния памяти
        try:
            allocated = torch.cuda.memory_allocated(0) / (1024**3)
            cached = torch.cuda.memory_reserved(0) / (1024**3)
            logger.info(f"🧹 Память после очистки: {allocated:.2f}GB выделено, {cached:.2f}GB кэшировано")
        except Exception as e:
            logger.warning(f"⚠️ Ошибка получения информации о памяти: {e}")
        
        logger.info("🚀 GPU память оптимизирована")

def detect_device_kaggle():
    """Улучшенное определение устройства для Kaggle"""
    # Проверка переменных окружения Kaggle
    is_kaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE') is not None
    kaggle_accelerator = os.environ.get('KAGGLE_ACCELERATOR', 'None')
    
    if is_kaggle:
        logger.info(f"🏃 Запуск в Kaggle, ускоритель: {kaggle_accelerator}")
    
    # Проверка доступности CUDA
    if torch.cuda.is_available() and torch.cuda.device_count() > 0:
        try:
            # Тестирование GPU
            device = torch.device('cuda:0')
            test_tensor = torch.randn(100, 100, device=device)
            _ = test_tensor @ test_tensor.T
            
            gpu_name = torch.cuda.get_device_name(0)
            gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
            
            logger.info(f"🔥 Используется GPU: {gpu_name}")
            logger.info(f"💾 GPU память: {gpu_memory:.2f} GB")
            
            # Очистка
            del test_tensor
            torch.cuda.empty_cache()
            
            # Оптимизация GPU
            optimize_gpu_memory()
            
            return {
                'device': 'cuda:0',
                'gpu_name': gpu_name,
                'gpu_memory_gb': gpu_memory,
                'is_kaggle': is_kaggle,
                'accelerator': kaggle_accelerator
            }
            
        except Exception as e:
            logger.warning(f"⚠️ GPU доступен, но не работает: {e}")
            return {
                'device': 'cpu',
                'gpu_name': None,
                'gpu_memory_gb': 0,
                'is_kaggle': is_kaggle,
                'accelerator': kaggle_accelerator
            }
    else:
        logger.info("💻 Используется CPU (GPU недоступен)")
        return {
            'device': 'cpu',
            'gpu_name': None,
            'gpu_memory_gb': 0,
            'is_kaggle': is_kaggle,
            'accelerator': kaggle_accelerator
        }

def get_optimal_config_for_device(device: str) -> dict:
    """Получение оптимальной конфигурации для устройства с учетом ограничений памяти T4 GPU"""
    if device.startswith('cuda'):
        # Проверяем количество доступных GPU
        gpu_count = torch.cuda.device_count() if torch.cuda.is_available() else 1
        
        # Получаем информацию о GPU памяти
        if torch.cuda.is_available():
            gpu_memory_gb = torch.cuda.get_device_properties(0).total_memory / (1024**3)
            logger.info(f"🔍 Обнаружено GPU: {torch.cuda.get_device_name(0)}, память: {gpu_memory_gb:.1f} GB")
        else:
            gpu_memory_gb = 0
        
        # Оптимизированные настройки для T4 GPU (15GB)
        if gpu_memory_gb <= 16:  # T4 или аналогичные GPU
            base_batch_size = 2  # Очень консервативный batch_size для больших моделей
            workers = 2
            cache_setting = False  # Отключаем кэширование для экономии памяти
        else:  # Более мощные GPU
            base_batch_size = 4
            workers = 4
            cache_setting = True
        
        # Увеличиваем batch_size пропорционально количеству GPU
        effective_batch_size = base_batch_size * max(1, gpu_count)
        
        logger.info(f"🎯 Настройки для {gpu_count} GPU: batch_size={effective_batch_size}, workers={workers}")
        
        return {
            'batch_size': effective_batch_size,
            'workers': workers,
            'mixed_precision': True,  # Включаем AMP для экономии памяти
            'cache': cache_setting,
            'optimizer': 'AdamW',
            'lr0': 0.001,
            'epochs': 100,
            'gpu_count': gpu_count
            # 'accumulate' удален - не поддерживается в Ultralytics YOLO
        }
    else:
        return {
            'batch_size': 2,
            'workers': 1,
            'mixed_precision': False,
            'cache': False,
            'optimizer': 'SGD',
            'lr0': 0.01,
            'epochs': 50,
            'gpu_count': 0
            # 'accumulate' удален - не поддерживается в Ultralytics YOLO
        }

def create_yolov13_config(num_classes: int = 1) -> dict:
    """Создание конфигурации YOLOv13 с динамической маршрутизацией"""
    return {
        'nc': num_classes,
        'depth_multiple': 0.67,
        'width_multiple': 0.75,
        'anchors': 3,
        
        # Backbone с интеграцией динамических модулей
        'backbone': [
            [-1, 1, 'Conv', [64, 6, 2, 2]],  # 0-P1/2
            [-1, 1, 'Conv', [128, 3, 2]],    # 1-P2/4
            [-1, 3, 'C2f', [128, True]],     # 2
            [-1, 1, 'Conv', [256, 3, 2]],    # 3-P3/8
            [-1, 6, 'C2f', [256, True]],     # 4
            [-1, 1, 'Conv', [512, 3, 2]],    # 5-P4/16
            [-1, 6, 'C2f', [512, True]],     # 6
            [-1, 1, 'Conv', [1024, 3, 2]],   # 7-P5/32
            [-1, 3, 'C2f', [1024, True]],    # 8
            [-1, 1, 'SPPF', [1024, 5]],      # 9
        ],
        
        # Head с улучшенной архитектурой для UAV
        'head': [
            [-1, 1, 'nn.Upsample', [None, 2, 'nearest']],  # 10
            [[-1, 6], 1, 'Concat', [1]],                    # 11
            [-1, 3, 'C2f', [512]],                          # 12
            [-1, 1, 'nn.Upsample', [None, 2, 'nearest']],   # 13
            [[-1, 4], 1, 'Concat', [1]],                    # 14
            [-1, 3, 'C2f', [256]],                          # 15
            [-1, 1, 'Conv', [256, 3, 2]],                   # 16
            [[-1, 12], 1, 'Concat', [1]],                   # 17
            [-1, 3, 'C2f', [512]],                          # 18
            [-1, 1, 'Conv', [512, 3, 2]],                   # 19
            [[-1, 9], 1, 'Concat', [1]],                    # 20
            [-1, 3, 'C2f', [1024]],                         # 21
            [[15, 18, 21], 1, 'Detect', [num_classes]],     # 22 Detect
        ]
    }

def train_yolov13_dynamic_uav(
    data_yaml: str = None,
    epochs: int = 100,
    batch_size: int = 16,
    img_size: int = 640,
    device: str = 'auto',
    project: str = 'yolov13_uav_runs',
    name: str = 'dynamic_routing_experiment',
    save_dir: str = './models',
    use_kaggle_datasets: bool = True,
    **kwargs
) -> Tuple[object, Dict]:
    """Обучение YOLOv13 с динамической маршрутизацией для UAV
    
    Args:
        data_yaml: Путь к YAML файлу с конфигурацией датасета (если None, используются Kaggle датасеты)
        epochs: Количество эпох обучения
        batch_size: Размер батча
        img_size: Размер входных изображений
        device: Устройство для обучения ('auto', 'cpu', 'cuda')
        project: Папка проекта для сохранения результатов
        name: Имя эксперимента
        save_dir: Директория для сохранения моделей
        use_kaggle_datasets: Использовать Kaggle датасеты вместо data_yaml
        **kwargs: Дополнительные параметры (включая debug_conf, debug_iou, debug_mode)
    
    Returns:
        Tuple[object, Dict]: Обученная модель и результаты обучения
    """
    
    # Автоматическое определение устройства
    if device == 'auto':
        device_info = detect_device_kaggle()
        device = device_info.get('device', 'cpu')
    else:
        # Если device передан явно, получаем информацию о нем
        device_info = detect_device_kaggle()
        device = device if device != 'auto' else device_info.get('device', 'cpu')
    
    # Получение оптимальной конфигурации
    device_config = get_optimal_config_for_device(device)
    
    # Адаптация параметров с учетом ограничений памяти
    batch_size = min(batch_size, device_config['batch_size'])
    epochs = min(epochs, device_config['epochs'])
    
    # Принудительно устанавливаем img_size в 1056 как требуется
    img_size = 1056
    logger.info(f"📐 Размер изображения установлен в {img_size} как требуется")
    
    # Настройка multi-GPU
    gpu_count = device_config.get('gpu_count', 1)
    if gpu_count > 1:
        device = f"0,1"  # Используем первые два GPU
        logger.info(f"🔥 Настройка multi-GPU: используем {gpu_count} GPU ({device})")
    
    # Градиентное накопление не поддерживается в Ultralytics YOLO
    # accumulate_steps удален для совместимости
    
    # Создание директорий
    os.makedirs(save_dir, exist_ok=True)
    os.makedirs(project, exist_ok=True)
    
    logger.info("🚁 YOLOv13 Dynamic UAV - Система обнаружения людей для БПЛА")
    logger.info("=" * 65)
    
    # Обработка режима отладки
    if IS_DEBUG:
        epochs = 2  # Используем 3 эпохи для отладки (минимум для получения значимых метрик)
        logger.info("🐛 РЕЖИМ ОТЛАДКИ АКТИВЕН")
        logger.info(f"   Эпохи: {epochs} (принудительно установлено для отладки)")
        logger.info(f"   Размер изображения: {img_size} (принудительно установлено)")
        logger.info(f"   Ограничение изображений: 800")
        logger.info(f"   Валидация: на всех данных")
    
    # Подготовка датасета
    if use_kaggle_datasets or data_yaml is None:
        logger.info("📊 Использование Kaggle датасетов")
        
        # Создание YAML конфигурации для объединенных датасетов
        yaml_path = create_combined_dataset_yaml('kaggle_combined_data.yaml')
        if not yaml_path:
            raise ValueError("❌ Не удалось создать YAML конфигурацию для датасетов")
        
        data_yaml = yaml_path
        logger.info(f"✅ Создана конфигурация датасета: {data_yaml}")
    
    logger.info(f"📊 Параметры обучения:")
    logger.info(f"   Датасет: {data_yaml}")
    logger.info(f"   Эпохи: {epochs}")
    logger.info(f"   Размер батча: {batch_size}")
    logger.info(f"   Размер изображения: {img_size}")
    logger.info(f"   Устройство: {device}")
    logger.info(f"   Режим отладки: {IS_DEBUG}")
    
    # Создание директории для кэша
    cache_dir = '/kaggle/working/cache'
    os.makedirs(cache_dir, exist_ok=True)
    logger.info(f"💽 Директория кэша: {cache_dir}")
    
    # Извлечение кастомных параметров из kwargs перед созданием training_config
    debug_conf = kwargs.pop('debug_conf', 0.25)
    debug_iou = kwargs.pop('debug_iou', 0.7)
    debug_mode = kwargs.pop('debug_mode', False)
    
    # Оптимизированная конфигурация для UAV с поддержкой multi-GPU и экономией памяти
    training_config = {
        'data': data_yaml,
        'epochs': epochs,
        'batch': min(batch_size + 2, 8) if debug_mode else batch_size,  # Увеличиваем batch для debug
        'imgsz': img_size,
        'device': [0,1],
        'project': project,
        'name': name,
        'save': True,
        'save_period': 1 if IS_DEBUG else 5,  # В debug режиме сохраняем каждую эпоху
        'patience': 20,
        'workers': device_config['workers'],
        'cache': device_config['cache'],  # Используем настройку из device_config
        
        # Градиентное накопление удалено - не поддерживается в Ultralytics YOLO
        
        # Оптимизатор
        'optimizer': device_config['optimizer'],
        'lr0': device_config['lr0'] * 0.5 if debug_mode else device_config['lr0'],  # Снижаем LR для debug
        'lrf': 0.01,
        'momentum': 0.937,
        'weight_decay': 0.0005,
        'warmup_epochs': 2 if debug_mode else 3,  # Увеличиваем warmup для debug
        'warmup_momentum': 0.8,
        'warmup_bias_lr': 0.05 if debug_mode else 0.1,  # Снижаем bias LR для debug
        'cos_lr': True,
        
        # Включаем AMP для экономии памяти
        'amp': device_config['mixed_precision'],
        
        # UAV-оптимизированные аугментации (уменьшенные для экономии памяти)
        'hsv_h': 0.01,  # Уменьшено
        'hsv_s': 0.5,   # Уменьшено
        'hsv_v': 0.3,   # Уменьшено
        'degrees': 10.0,  # Уменьшено
        'translate': 0.05,  # Уменьшено
        'scale': 0.3,   # Уменьшено
        'shear': 1.0,   # Уменьшено
        'perspective': 0.0,
        'flipud': 0.0,
        'fliplr': 0.5,
        'mosaic': 0.5 if debug_mode else 0.8,  # Снижаем для debug
        'mixup': 0.1,   # Уменьшено
        'copy_paste': 0.0 if debug_mode else 0.1,  # Отключаем для debug
        
        # Оптимизированные веса потерь для людей
        'box': 5.0 if debug_mode else 7.5,  # Снижаем box loss для debug
        'cls': 1.0 if debug_mode else 0.5,   # Увеличиваем cls loss для debug
        'dfl': 1.5,
        
        # NMS параметры для инференса
        'iou': 0.7,
        'conf': 0.25,
        
        # Параметры валидации
        'val': True,
        'plots': True,
        'rect': False,
        
        # Дополнительные параметры для экономии памяти
        'close_mosaic': 10,  # Отключаем мозаику на последних эпохах
        'deterministic': False,  # Для лучшей производительности
        'single_cls': True,  # У нас только один класс (человек)
        
        # Дополнительные параметры для стабилизации обучения
        'label_smoothing': 0.0 if debug_mode else 0.1,  # Отключаем label smoothing для debug
        'nbs': 64,  # Nominal batch size для нормализации
        'overlap_mask': True,  # Разрешаем перекрывающиеся маски
        'mask_ratio': 4,  # Соотношение для масок сегментации
        
        # Исключаем **kwargs чтобы избежать передачи недопустимых параметров в YOLO
    }
    
    try:
        # Инициализация базовой YOLO модели
        logger.info("🔄 Загрузка базовой YOLO модели...")
        base_model = YOLO('yolo12l.pt')
        
        # Создание конфигурации YOLOv13
        yolov13_config = create_yolov13_config(num_classes=1)
        
        # Сохранение конфигурации
        config_path = os.path.join(save_dir, 'yolov13_dynamic_uav.yaml')
        with open(config_path, 'w') as f:
            yaml.dump(yolov13_config, f, default_flow_style=False)
        
        logger.info(f"💾 Конфигурация YOLOv13 сохранена: {config_path}")
        
        # Логирование оптимизированных параметров
        logger.info("🔧 Оптимизированные параметры обучения:")
        logger.info(f"   Финальный размер батча: {training_config['batch']}")
        logger.info(f"   Warmup эпохи: {training_config['warmup_epochs']}")
        logger.info(f"   Loss веса - box: {training_config['box']}, cls: {training_config['cls']}, dfl: {training_config['dfl']}")
        logger.info(f"   Аугментации - mosaic: {training_config['mosaic']}, copy_paste: {training_config['copy_paste']}")
        logger.info(f"   Режим отладки активен: {debug_mode}")
        
        # Обучение модели
        logger.info("🚀 Начало обучения YOLOv13 Dynamic UAV...")
        start_time = time.time()
        
        results = base_model.train(**training_config)
        
        training_time = time.time() - start_time
        logger.info(f"⏱️ Время обучения: {training_time/60:.2f} минут")
        
        # Сохранение лучшей модели с улучшенной логикой поиска
        best_model_path = os.path.join(save_dir, 'yolov13_dynamic_uav_best.pt')
        model_copied = False
        
        # Создаем папку models если её нет
        os.makedirs(save_dir, exist_ok=True)
        logger.info(f"📁 Папка для сохранения модели: {save_dir}")
        
        # Попытка 1: Стандартный путь через results.save_dir
        if hasattr(results, 'save_dir') and results.save_dir:
            weights_dir = Path(results.save_dir) / 'weights'
            best_path = weights_dir / 'best.pt'
            logger.info(f"🔍 Поиск модели по пути: {best_path}")
            
            if best_path.exists():
                try:
                    shutil.copy2(best_path, best_model_path)
                    logger.info(f"✅ Лучшая модель сохранена: {best_model_path}")
                    model_copied = True
                except Exception as e:
                    logger.error(f"❌ Ошибка копирования модели: {e}")
            else:
                logger.warning(f"⚠️ Файл best.pt не найден по пути: {best_path}")
        
        # Попытка 2: Поиск в папке проекта
        if not model_copied:
            project_dir = Path(project) if project else Path('yolov13_uav_human_detection')
            if project_dir.exists():
                # Ищем последнюю папку с обучением
                train_dirs = [d for d in project_dir.iterdir() if d.is_dir() and 'kaggle_training' in d.name]
                if train_dirs:
                    latest_dir = max(train_dirs, key=lambda x: x.stat().st_mtime)
                    best_path = latest_dir / 'weights' / 'best.pt'
                    logger.info(f"🔍 Поиск модели в последней папке обучения: {best_path}")
                    
                    if best_path.exists():
                        try:
                            shutil.copy2(best_path, best_model_path)
                            logger.info(f"✅ Лучшая модель найдена и сохранена: {best_model_path}")
                            model_copied = True
                        except Exception as e:
                            logger.error(f"❌ Ошибка копирования модели: {e}")
        
        # Попытка 3: Поиск в текущей директории
        if not model_copied:
            current_dir = Path('.')
            best_files = list(current_dir.rglob('best.pt'))
            if best_files:
                # Берем самый новый файл
                latest_best = max(best_files, key=lambda x: x.stat().st_mtime)
                logger.info(f"🔍 Найден файл best.pt: {latest_best}")
                try:
                    shutil.copy2(latest_best, best_model_path)
                    logger.info(f"✅ Лучшая модель найдена и сохранена: {best_model_path}")
                    model_copied = True
                except Exception as e:
                    logger.error(f"❌ Ошибка копирования модели: {e}")
        
        if not model_copied:
            logger.error(f"❌ Не удалось найти и скопировать модель best.pt")
            logger.info(f"🔍 Проверьте наличие файла в папках обучения")
        
        # Валидация модели
        logger.info("📊 Валидация модели...")
        
        # Дополнительное подавление matplotlib warnings перед валидацией
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", RuntimeWarning)
            warnings.filterwarnings('ignore', message='invalid value encountered in less')
            warnings.filterwarnings('ignore', message='invalid value encountered in greater')
            warnings.filterwarnings('ignore', message='divide by zero encountered')
            warnings.filterwarnings('ignore', category=UserWarning, module='matplotlib')
            
            # Выбор параметров валидации в зависимости от режима
            if debug_mode:
                val_conf = debug_conf
                val_iou = debug_iou
                logger.info(f"🐛 Режим отладки: используем conf={val_conf}, iou={val_iou}")
            else:
                val_conf = 0.001  # Низкий порог уверенности для лучшего recall
                val_iou = 0.6     # Оптимальный IoU порог
                logger.info(f"🎯 Обычный режим: используем conf={val_conf}, iou={val_iou}")
        
            try:
                # Подавление matplotlib предупреждений во время валидации
                with warnings.catch_warnings():
                    warnings.filterwarnings('ignore', category=RuntimeWarning, module='matplotlib')
                    warnings.filterwarnings('ignore', message='invalid value encountered in less')
                    warnings.filterwarnings('ignore', message='invalid value encountered in greater')
                    warnings.filterwarnings('ignore', message='divide by zero encountered')
                    warnings.filterwarnings('ignore', category=UserWarning, module='matplotlib')
                    
                    # Валидация с выбранными параметрами
                    val_results = base_model.val(
                        data=data_yaml,
                        conf=val_conf,
                        iou=val_iou,
                        verbose=True
                    )
                if hasattr(val_results, 'box'):
                    try:
                        map50 = getattr(val_results.box, 'map50', None)
                        map_val = getattr(val_results.box, 'map', None)
                        precision = getattr(val_results.box, 'mp', None)
                        recall = getattr(val_results.box, 'mr', None)
                        
                        logger.info("📈 Результаты валидации:")
                        if map50 is not None and not math.isnan(map50):
                            logger.info(f"   mAP50: {map50:.4f}")
                        else:
                            logger.warning("   mAP50: не вычислен")
                        
                        if map_val is not None and not math.isnan(map_val):
                            logger.info(f"   mAP50-95: {map_val:.4f}")
                        else:
                            logger.warning("   mAP50-95: не вычислен")
                        
                        if precision is not None and not math.isnan(precision):
                            logger.info(f"   Precision: {precision:.4f}")
                        else:
                            logger.warning("   Precision: не вычислен")
                        
                        if recall is not None and not math.isnan(recall):
                            logger.info(f"   Recall: {recall:.4f}")
                        else:
                            logger.warning("   Recall: не вычислен")
                            
                    except Exception as e:
                        logger.error(f"❌ Ошибка обработки результатов валидации: {e}")
                else:
                    logger.warning("⚠️ Результаты валидации не содержат метрики box")
            except Exception as e:
                logger.warning(f"⚠️ Ошибка валидации: {e}")
                val_results = None
        
        # Валидация на приватном датасете (если доступен)
        if os.path.exists(best_model_path):
            logger.info("🔒 Валидация на приватном датасете...")
            private_val_results = validate_on_private_dataset(
                best_model_path, 
                debug_conf=debug_conf, 
                debug_iou=debug_iou, 
                debug_mode=debug_mode
            )
            if 'error' not in private_val_results:
                logger.info("✅ Приватная валидация завершена успешно")
            else:
                logger.warning(f"⚠️ Ошибка приватной валидации: {private_val_results['error']}")
        
        # Экспорт модели
        logger.info("📦 Экспорт модели...")
        export_formats = ['onnx']
        if device.startswith('cuda'):
            export_formats.append('torchscript')
        
        for fmt in export_formats:
            try:
                export_path = base_model.export(
                    format=fmt,
                    imgsz=img_size,
                    optimize=True,
                    half=device.startswith('cuda')
                )
                logger.info(f"✅ Модель экспортирована в {fmt}: {export_path}")
            except Exception as e:
                logger.warning(f"⚠️ Ошибка экспорта в {fmt}: {e}")
        
        # Создание отчета
        report = {
            'model_path': best_model_path,
            'best_model_path': best_model_path,  # Явно указываем путь к лучшей модели
            'config_path': config_path,
            'training_config': training_config,
            'training_time_minutes': training_time / 60,
            'results': val_results.results_dict if val_results and hasattr(val_results, 'results_dict') else {},
            'training_completed': True,
            'success': True,  # Добавляем флаг успеха
            'device_used': device,
            'final_epochs': epochs,
            'final_batch_size': batch_size,
            'final_img_size': img_size,  # Добавляем финальный размер изображения
            'yolov13_features': {
                'dynamic_routing': True,
                'uav_optimization': True,
                'human_detection': True,
                'multi_scale_features': True
            },
            'best_metrics': {}  # Инициализируем пустой словарь для метрик
        }
        
        logger.info("\n🎉 Обучение YOLOv13 Dynamic UAV завершено успешно!")
        logger.info(f"💾 Модель сохранена: {best_model_path}")
        logger.info(f"🎯 Модель готова для обнаружения людей с UAV!")
        
        return base_model, report
        
    except Exception as e:
        logger.error(f"❌ Ошибка во время обучения: {e}")
        import traceback
        traceback.print_exc()
        
        # Безопасное создание отчета об ошибке
        error_report = {
            'error': str(e),
            'training_completed': False,
            'success': False,
            'device_used': device if 'device' in locals() else 'unknown'
        }
        
        # Добавляем конфигурацию только если она была создана
        if 'training_config' in locals():
            error_report['config'] = training_config
            
        return None, error_report

def prepare_uav_dataset(dataset_path: str, output_path: str) -> str:
    """Подготовка датасета для UAV обнаружения людей"""
    
    # Создание структуры директорий
    dirs = {
        'train_img': os.path.join(output_path, "train/images"),
        'train_lbl': os.path.join(output_path, "train/labels"),
        'val_img': os.path.join(output_path, "val/images"),
        'val_lbl': os.path.join(output_path, "val/labels")
    }
    
    for dir_path in dirs.values():
        os.makedirs(dir_path, exist_ok=True)
    
    # Проверка существования датасета
    images_path = os.path.join(dataset_path, "images")
    labels_path = os.path.join(dataset_path, "labels")
    
    if not os.path.exists(images_path):
        logger.warning(f"⚠️ Директория изображений не найдена: {images_path}")
        # Создание тестового data.yaml
        return create_test_data_yaml(output_path)
    
    # Получение списка изображений
    image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff')
    image_files = [f for f in os.listdir(images_path) 
                  if f.lower().endswith(image_extensions)]
    
    if len(image_files) == 0:
        logger.warning("⚠️ Изображения не найдены")
        return create_test_data_yaml(output_path)
    
    # Разделение на train/val (80/20)
    train_files, val_files = train_test_split(
        image_files, test_size=0.2, random_state=42
    )
    
    def copy_dataset_files(files, split_type):
        copied_count = 0
        for file in files:
            # Копирование изображения
            img_src = os.path.join(images_path, file)
            img_dst = os.path.join(dirs[f'{split_type}_img'], file)
            
            if os.path.exists(img_src):
                shutil.copy2(img_src, img_dst)
                copied_count += 1
                
                # Копирование соответствующего файла меток
                label_file = os.path.splitext(file)[0] + '.txt'
                label_src = os.path.join(labels_path, label_file)
                label_dst = os.path.join(dirs[f'{split_type}_lbl'], label_file)
                
                if os.path.exists(label_src):
                    # Проверка и корректировка меток для класса "человек"
                    with open(label_src, 'r') as f:
                        lines = f.readlines()
                    
                    corrected_lines = []
                    for line in lines:
                        line = line.strip()
                        if line:
                            parts = line.split()
                            if len(parts) >= 5:
                                try:
                                    # Валидация координат
                                    x_center, y_center, width, height = map(float, parts[1:5])
                                    if (0 <= x_center <= 1 and 0 <= y_center <= 1 and 
                                        0 <= width <= 1 and 0 <= height <= 1):
                                        # Изменяем класс на 0 (человек) если это необходимо
                                        parts[0] = '0'
                                        corrected_lines.append(' '.join(parts))
                                    else:
                                        logger.warning(f"⚠️ Некорректные координаты в {label_file}: {parts[1:5]}")
                                except ValueError:
                                    logger.warning(f"⚠️ Ошибка парсинга координат в {label_file}: {parts[1:5]}")
                    
                    with open(label_dst, 'w') as f:
                        if corrected_lines:
                            f.write('\n'.join(corrected_lines))
                        else:
                            f.write('')  # Пустой файл если нет валидных аннотаций
                else:
                    # Создание пустого файла меток
                    with open(label_dst, 'w') as f:
                        f.write('')
        
        return copied_count
    
    train_count = copy_dataset_files(train_files, "train")
    val_count = copy_dataset_files(val_files, "val")
    
    # Создание data.yaml
    data_yaml_content = f"""
path: {output_path}
train: train/images
val: val/images

nc: 1
names: ['human']
"""
    
    data_yaml_path = os.path.join(output_path, 'data.yaml')
    with open(data_yaml_path, 'w') as file:
        file.write(data_yaml_content)
    
    logger.info(f"✅ Датасет подготовлен: {train_count} train, {val_count} val")
    logger.info(f"📁 Конфигурация датасета: {data_yaml_path}")
    
    return data_yaml_path

def create_test_data_yaml(output_path: str) -> str:
    """Создание тестового data.yaml файла"""
    data_yaml_content = f"""
path: {output_path}
train: train/images
val: val/images

nc: 1
names: ['human']
"""
    
    data_yaml_path = os.path.join(output_path, 'data.yaml')
    with open(data_yaml_path, 'w') as file:
        file.write(data_yaml_content)
    
    logger.info(f"📝 Создан тестовый data.yaml: {data_yaml_path}")
    return data_yaml_path

def main():
    """Основная функция для запуска обучения YOLOv13 с Kaggle датасетами"""
    logger.info("🚁 YOLOv13 Dynamic UAV - Система обнаружения людей для БПЛА")
    logger.info("=" * 70)
    # Очистка временных debug файлов
    if IS_DEBUG:
        cleanup_debug_files()
    
    # Проверка режима отладки
    if IS_DEBUG:
        logger.info("🐛 РЕЖИМ ОТЛАДКИ АКТИВЕН")
        logger.info("   • Обучение на 800 изображениях")
        logger.info("   • 3 эпохи обучения (оптимизировано для отладки)")
        logger.info("   • Валидация на всех данных")
        logger.info("   • Пониженные пороги для детекции (conf=0.15, iou=0.6)")
    else:
        logger.info("🎯 ПОЛНЫЙ РЕЖИМ ОБУЧЕНИЯ")
        logger.info("   • Обучение на всех доступных данных")
        logger.info("   • Стандартное количество эпох")
    
    # Создание директорий
    data_path = Path("./data")
    models_path = Path("./models")
    
    data_path.mkdir(exist_ok=True)
    models_path.mkdir(exist_ok=True)
    
    # Проверка доступности основных датасетов
    logger.info("\n📊 Проверка доступности Kaggle датасетов...")
    
    # Быстрая проверка существования основных датасетов
    train_datasets_exist = os.path.exists(TRAIN_DATASET_1) or os.path.exists(TRAIN_DATASET_2)
    val_public_exists = os.path.exists(VAL_DATASET_PUBLIC)
    
    if not train_datasets_exist:
        logger.error("❌ Не найдено ни одного тренировочного датасета!")
        logger.error("   Проверьте пути к датасетам:")
        logger.error(f"   • {TRAIN_DATASET_1}")
        logger.error(f"   • {TRAIN_DATASET_2}")
        return None, {'error': 'Тренировочные датасеты не найдены'}
    
    if not val_public_exists:
        logger.error("❌ Не найден публичный валидационный датасет!")
        logger.error(f"   Проверьте путь: {VAL_DATASET_PUBLIC}")
        return None, {'error': 'Публичный валидационный датасет не найден'}
    
    logger.info("✅ Основные датасеты найдены")
    logger.info(f"   • Тренировочные: {TRAIN_DATASET_1 if os.path.exists(TRAIN_DATASET_1) else ''} {TRAIN_DATASET_2 if os.path.exists(TRAIN_DATASET_2) else ''}")
    logger.info(f"   • Валидационный публичный: {VAL_DATASET_PUBLIC}")
    logger.info(f"   • Валидационный приватный: {'✅' if os.path.exists(VAL_DATASET_PRIVATE) else '❌'}")
    
    # Оптимизация GPU памяти перед началом обучения
    optimize_gpu_memory()
    
    # Определение устройства и оптимального размера изображения
    device_info = detect_device_kaggle()
    device_str = device_info.get('device', 'cpu')
    
    # Расчет оптимального размера изображения на основе доступной памяти GPU
    base_img_size = 1056 if IS_DEBUG else 832  # Размер 1056 для debug режима как требуется
    optimal_img_size = calculate_optimal_img_size(device_str, base_img_size)
    
    logger.info(f"🎯 Оптимальный размер изображения: {optimal_img_size}x{optimal_img_size}")
    
    # Параметры обучения
    training_params = {
        'data_yaml': None,  # Будет создан автоматически из Kaggle датасетов
        'epochs': 2 if IS_DEBUG else 1000,  # 3 эпохи для режима отладки
        'batch_size': 8 if IS_DEBUG else 16,
        'img_size': 1056,  # Фиксированный размер изображения 1056 как требуется
        'device': 'auto',
        'save_dir': models_path,
        'project': 'yolov13_uav_human_detection',
        'name': f'kaggle_training_debug_{int(time.time())}' if IS_DEBUG else f'kaggle_training_{int(time.time())}',
        'use_kaggle_datasets': True,
        # Специальные параметры для режима отладки
        'debug_conf': 0.15 if IS_DEBUG else 0.25,  # Умеренный порог уверенности для отладки
        'debug_iou': 0.6 if IS_DEBUG else 0.7,     # Умеренный NMS для отладки
        'debug_mode': IS_DEBUG
    }
    
    logger.info(f"\n📋 Параметры обучения:")
    for key, value in training_params.items():
        logger.info(f"   {key}: {value}")
    
    try:
        # Запуск обучения
        logger.info("\n🚀 Запуск обучения...")
        model, results = train_yolov13_dynamic_uav(**training_params)
        
        if results.get('success', False):
            logger.info("\n🎉 Обучение завершено успешно!")
            logger.info(f"📁 Модель сохранена: {results['model_path']}")
            
            # Отображение метрик с проверками
            best_metrics = results.get('best_metrics', {})
            if best_metrics:
                logger.info(f"📊 Лучшие метрики обучения:")
                for metric, value in best_metrics.items():
                    try:
                        if value is not None and isinstance(value, (int, float)) and not math.isnan(value):
                            logger.info(f"   {metric}: {value:.4f}")
                        elif value is not None:
                            logger.info(f"   {metric}: {value}")
                        else:
                            logger.warning(f"   {metric}: None (метрика не вычислена)")
                    except Exception as e:
                        logger.error(f"   ❌ Ошибка отображения метрики {metric}: {e}")
            else:
                logger.warning("⚠️ Метрики обучения не найдены в результатах")
            
            # Тестирование модели
            try:
                logger.info("\n🧪 Тестирование обученной модели...")
                
                # Поиск лучшей модели с улучшенной логикой
                best_model_path = None
                search_paths = []
                
                # 1. Проверяем best_model_path из результатов
                if 'best_model_path' in results and results['best_model_path']:
                    search_paths.append(results['best_model_path'])
                
                # 2. Проверяем model_path из результатов
                if 'model_path' in results and results['model_path']:
                    search_paths.append(results['model_path'])
                
                # 3. Проверяем стандартный путь в models/
                standard_model_path = os.path.join(models_path, 'yolov13_dynamic_uav_best.pt')
                search_paths.append(standard_model_path)
                
                # 4. Проверяем в директории проекта
                if 'save_dir' in results and results['save_dir']:
                    project_best = os.path.join(results['save_dir'], 'weights', 'best.pt')
                    search_paths.append(project_best)
                    project_last = os.path.join(results['save_dir'], 'weights', 'last.pt')
                    search_paths.append(project_last)
                
                # 5. Поиск в последней папке kaggle_training
                try:
                    project_dir = 'yolov13_uav_human_detection'
                    if os.path.exists(project_dir):
                        kaggle_dirs = [d for d in os.listdir(project_dir) if d.startswith('kaggle_training')]
                        if kaggle_dirs:
                            latest_dir = max(kaggle_dirs, key=lambda x: os.path.getctime(os.path.join(project_dir, x)))
                            latest_best = os.path.join(project_dir, latest_dir, 'weights', 'best.pt')
                            latest_last = os.path.join(project_dir, latest_dir, 'weights', 'last.pt')
                            search_paths.extend([latest_best, latest_last])
                except Exception as e:
                    logger.warning(f"⚠️ Ошибка поиска в kaggle_training директориях: {e}")
                
                # Поиск существующей модели
                logger.info(f"🔍 Поиск модели в {len(search_paths)} возможных местах...")
                for i, path in enumerate(search_paths, 1):
                    logger.debug(f"   {i}. Проверяем: {path}")
                    if path and os.path.exists(path):
                        best_model_path = path
                        logger.info(f"✅ Найдена модель: {best_model_path}")
                        break
                
                if not best_model_path:
                    logger.error("❌ Модель не найдена ни в одном из возможных мест:")
                    for i, path in enumerate(search_paths, 1):
                        logger.error(f"   {i}. {path} - {'существует' if path and os.path.exists(path) else 'не существует'}")
                    raise FileNotFoundError(f"Model not found in any of the search paths")
                
                # test_model = YOLO(best_model_path)
                logger.info("✅ Модель успешно загружена")
                logger.info("🎯 Готова для обнаружения людей с UAV")
                
                # Финальная валидация на приватном датасете с интегрированными метриками
                if os.path.exists(VAL_DATASET_PRIVATE):
                    logger.info("\n🔒 Финальная валидация на приватном датасете...")
                    
                    try:
                        # Обработка изображений с помощью адаптированной функции
                        logger.info("📊 Обработка изображений для расчета метрик...")
                        predicted_df = process_images_adapted(
                            VAL_DATASET_PRIVATE,
                            result_csv_path=os.path.join(log_dir, 'private_predictions.csv')
                        )
                        
                        if not predicted_df.empty:
                            logger.info(f"✅ Обработано {len(predicted_df)} предсказаний")
                            
                            # Конвертируем DataFrame в bytes для функции evaluate
                            predicted_bytes = df_to_bytes(predicted_df)
                            
                            # Проверяем наличие ground truth файла
                            gt_csv_path = os.path.join(VAL_DATASET_PRIVATE, 'gt.csv')
                            if not os.path.exists(gt_csv_path):
                                # Попробуем альтернативные пути
                                alternative_gt_paths = [
                                    os.path.join(VAL_DATASET_PRIVATE, 'ground_truth.csv'),
                                    os.path.join(VAL_DATASET_PRIVATE, 'labels.csv'),
                                    PUBLIC_GT_CSV_PATH
                                ]
                                for alt_path in alternative_gt_paths:
                                    if os.path.exists(alt_path):
                                        gt_csv_path = alt_path
                                        break
                                else:
                                    logger.warning("⚠️ Ground truth файл не найден, создаем пустой для демонстрации")
                                    # Создаем минимальный GT файл для демонстрации
                                    empty_gt = pd.DataFrame(columns=COLUMNS)
                                    gt_csv_path = os.path.join(log_dir, 'empty_gt.csv')
                                    empty_gt.to_csv(gt_csv_path, index=False)
                            
                            # Загружаем ground truth
                            gt_bytes = open_df_as_bytes(gt_csv_path)
                            
                            # Вычисляем метрики с помощью функции evaluate
                            logger.info("🧮 Расчет финальных метрик...")
                            metric, accuracy, fp_rate, avg_time = evaluate(
                                predicted_file=predicted_bytes,
                                gt_file=gt_bytes,
                                thresholds=np.round(np.arange(0.3, 1.0, 0.07), 2),
                                beta=1.0,
                                m=500,
                                parallelize=True
                            )
                            
                            # Логируем результаты
                            logger.info("📊 Результаты финальной валидации:")
                            logger.info(f"   🎯 Общая метрика: {metric:.4f}")
                            logger.info(f"   ⏱️ Среднее время обработки: {avg_time:.4f}с")
                            
                            if accuracy:
                                logger.info("   📈 Точность по порогам:")
                                for threshold, acc in accuracy.items():
                                    logger.info(f"      Порог {threshold}: {acc:.4f}")
                            
                            if fp_rate:
                                logger.info("   ⚠️ Частота ложных срабатываний:")
                                for threshold, fpr in fp_rate.items():
                                    logger.info(f"      Порог {threshold}: {fpr:.4f}")
                            
                            # Дополнительная валидация с помощью существующей функции
                            final_private_results = validate_on_private_dataset(
                                best_model_path,
                                debug_conf=training_params.get('debug_conf', 0.25),
                                debug_iou=training_params.get('debug_iou', 0.7),
                                debug_mode=training_params.get('debug_mode', False)
                            )
                            
                            if 'error' not in final_private_results:
                                logger.info("📊 Дополнительные результаты валидации:")
                                for metric_name, value in final_private_results.items():
                                    try:
                                        if value is not None and isinstance(value, (int, float)) and not math.isnan(value):
                                            logger.info(f"   {metric_name}: {value:.4f}")
                                        elif value is not None:
                                            logger.info(f"   {metric_name}: {value}")
                                        else:
                                            logger.warning(f"   {metric_name}: None (метрика не вычислена)")
                                    except Exception as e:
                                        logger.error(f"   ❌ Ошибка отображения метрики {metric_name}: {e}")
                            else:
                                logger.warning(f"⚠️ Ошибка дополнительной валидации: {final_private_results['error']}")
                        
                        else:
                            logger.warning("⚠️ Не удалось получить предсказания для валидации")
                            
                    except Exception as e:
                        logger.error(f"❌ Ошибка при финальной валидации: {e}")
                        import traceback
                        traceback.print_exc()
                        
                else:
                    logger.warning(f"⚠️ Приватный датасет не найден: {VAL_DATASET_PRIVATE}")
                
            except Exception as e:
                logger.error(f"❌ Ошибка тестирования модели: {e}")
            
            # Рекомендации по использованию
            logger.info("\n📋 Рекомендации по использованию модели:")
            logger.info("   • Оптимальная высота съемки: 50-150м")
            logger.info("   • Рекомендуемое разрешение: 1920x1080 или выше")
            logger.info("   • Скорость полета: не более 15 м/с")
            logger.info("   • Угол камеры: 45-90° вниз")
            logger.info("   • Лучшие условия: дневное освещение, ясная погода")
            logger.info("   • Использование: model.predict('path_to_uav_image.jpg')")
            
            if IS_DEBUG:
                logger.info("\n🐛 Режим отладки завершен.")
                logger.info("   Для полного обучения установите IS_DEBUG = False")
                logger.info("   и перезапустите скрипт.")
            
            return model, results
            
        else:
            logger.error(f"\n❌ Обучение не завершено: {results.get('error', 'Неизвестная ошибка')}")
            return None, results
            
    except Exception as e:
        logger.error(f"\n💥 Критическая ошибка: {e}")
        import traceback
        traceback.print_exc()
        return None, {'error': str(e)}
        
    finally:
        
        
        logger.info("\n" + "=" * 70)
        logger.info("🏁 Завершение работы программы")

if __name__ == "__main__":
    # Запуск основной функции обучения YOLOv13 с Kaggle датасетами
    model, results = main()

2025-08-01 13:58:16,546 - MAIN - INFO - <cell line: 0>:97 - Logging setup completed
2025-08-01 13:58:16,558 - MAIN - INFO - main:1725 - 🚁 YOLOv13 Dynamic UAV - Система обнаружения людей для БПЛА
2025-08-01 13:58:16,559 - MAIN - INFO - main:1726 - ======================================================================
2025-08-01 13:58:16,561 - MAIN - WARNING - cleanup_debug_files:1002 - ⚠️ Ошибка очистки debug файлов: [Errno 16] Device or resource busy: '/kaggle/working'
2025-08-01 13:58:16,561 - MAIN - INFO - main:1733 - 🐛 РЕЖИМ ОТЛАДКИ АКТИВЕН
2025-08-01 13:58:16,562 - MAIN - INFO - main:1734 -    • Обучение на 800 изображениях
2025-08-01 13:58:16,563 - MAIN - INFO - main:1735 -    • 1 эпоха обучения
2025-08-01 13:58:16,564 - MAIN - INFO - main:1736 -    • Валидация на всех данных
2025-08-01 13:58:16,565 - MAIN - INFO - main:1750 - 
📊 Анализ доступных Kaggle датасетов...
2025-08-01 13:58:16,566 - MAIN - INFO - analyze_all_datasets:1015 - 📊 Анализ всех датасетов...
2025-08-01 13:58:16,5

[DEBUG] Logging setup completed successfully.


2025-08-01 13:58:34,553 - MAIN - INFO - collect_hierarchical_images:253 - ✅ Собрано 8484 пар изображение-метка из /kaggle/input/01trains1datasethumanrescu1
